# Test Microphone Track

In [2]:
from aiortc.contrib.media import MediaPlayer
import pyaudio

# PyAudio configuration
FORMAT = pyaudio.paInt16  # Adjust this based on the audio format
CHANNELS = 2  # Stereo
RATE = 48000  # Adjust this based on the audio sample rate

microphone_track=MediaPlayer("audio=Microphone Array (Realtek(R) Audio)", format="dshow")

audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    output=True)

while True:
    # await asyncio.sleep(0.1)
    frame=await microphone_track.recv()
    # frame=await microphone_track.recv()
    if frame:
        # print(frame)
        audio_data=frame.to_ndarray().tobytes()
        stream.write(audio_data)

ValueError: no container format 'dshow'

# Find audio samplerate, no. of channels

In [3]:
import wave

# Welcome Audio that has to be played in the starting
with wave.open("/home/oem/webrtc/src/callbot/audios/global_ime_1.wav", 'rb') as wav_file:
    # Get the number of channels
    channels = wav_file.getnchannels()

    # Get the sample rate
    sample_rate = wav_file.getframerate()

    # Get the sample width
    sample_width = wav_file.getsampwidth()

    # Get the number of frames
    num_frames = wav_file.getnframes()

    # Read the audio data
    audio_data = wav_file.readframes(num_frames)

print(f"Channels: {channels}")
print(f"Sample Rate: {sample_rate} Hz")
print(f"Sample Width: {sample_width} bytes")
# print(audio_data)

Channels: 1
Sample Rate: 24000 Hz
Sample Width: 2 bytes


In [2]:
import wave

# Welcome Audio that has to be played in the starting
with wave.open("/home/oem/webrtc/src/callbot/audios/e1.wav", 'rb') as wav_file:
    # Get the number of channels
    channels = wav_file.getnchannels()

    # Get the sample rate
    sample_rate = wav_file.getframerate()

    # Get the sample width
    sample_width = wav_file.getsampwidth()

    # Get the number of frames
    num_frames = wav_file.getnframes()

    # Read the audio data
    audio_data = wav_file.readframes(num_frames)

print(f"Channels: {channels}")
print(f"Sample Rate: {sample_rate} Hz")
print(f"Sample Width: {sample_width} bytes")
# print(audio_data)

Channels: 1
Sample Rate: 24000 Hz
Sample Width: 2 bytes


#  Call your phone


In [5]:
import wave
import time
from pyVoIP import VoIPClient, VoIPPhone, VoIPCall, CallState

phone = VoIPPhone(
        server="192.168.88.5",
        port=5060,
        username="5001",
        password="iX3TxsD9jWxmZU5",
        myIP="192.168.88.10"
    )
phone.call('9868205040')
# Wait for the call to be answered
while call.state not in [CallState.ANSWERED, CallState.FAILED]:
    time.sleep(1)  # Wait for 1 second before checking again

if call.state == CallState.ANSWERED:
    print("Call answered! Recording started.")
    # Implement your call handling logic here
    input("Press Enter to end the call and save the recording...")
elif call.state == CallState.FAILED:
    print("Call failed to connect.")

# End the call and close the wave file
phone.hangup()
wave_file.close()

AttributeError: FAILED

# Converting from 8-bit audio to 16-bit audio

In [11]:
import wave
import numpy as np
import torch
import torchaudio

def getAudioArray(path, sample_rate, channels, to_sample_rate, to_bit_depth=2):
    with wave.open(path, 'rb') as wav_file:
        num_frames = wav_file.getnframes()
        audio_data = wav_file.readframes(num_frames)

    # Convert to numpy array of appropriate bit depth
    np_array = np.frombuffer(audio_data, dtype=np.int16)
    np_array = np_array.astype(np.float32) / 32768.0

    # Handle 8-bit conversion if needed
    if to_bit_depth == 1:
        # Convert 16-bit audio to 8-bit
        np_array = (np_array * 127.5).astype(np.int8) / 128.0

    if channels == 2:
        return np_array
    elif channels == 1:
        audio_tensor = torch.from_numpy(np_array)
        audio_tensor = torchaudio.functional.resample(audio_tensor, orig_freq=sample_rate, new_freq=to_sample_rate)
        return audio_tensor.numpy()

sip_state1_array=getAudioArray("/home/oem/webrtc/src/callbot/audios/global_ime_1.wav",sample_rate=24000, channels=1, to_sample_rate=8000, to_bit_depth=1)

In [15]:
with wave.open('output_wave.wav', 'w') as wf:
        # Set parameters
        n_channels = 1  # Mono
        sampwidth = 1  # 2 bytes (16 bits)
        wf.setnchannels(n_channels)
        wf.setsampwidth(sampwidth)
        wf.setframerate(8000)
        
        # Write the data
        wf.writeframes((sip_state1_array*32768).astype(np.int8).tobytes())

In [26]:
import numpy as np
import scipy.io.wavfile as wav
from scipy.signal import resample

def get_audio_bytes(input_path):
    # Load the audio file
    sample_rate, data = wav.read(input_path)

    # Check if the audio is in 16-bit PCM format
    if data.dtype != np.int16:
        raise ValueError("Input audio is not in 16-bit PCM format")

    # Resample the audio to 8000 Hz
    target_sample_rate = 8000
    num_samples = int(len(data) * target_sample_rate / sample_rate)
    resampled_data = resample(data, num_samples)

    # Convert to 8-bit PCM
    min_val, max_val = np.min(resampled_data), np.max(resampled_data)
    # Scale the data to 8-bit range
    resampled_data = np.clip((resampled_data - min_val) / (max_val - min_val) * 255, 0, 255).astype(np.uint8)
    
    return resampled_data.tobytes()

# Usage
get_audio_bytes('/home/oem/webrtc/src/callbot/audios/global_ime_1.wav')


b'oooooppooooonooonooonnooonnoonnoonoooooonnooonnoonnnnnnnnmnmlmmmmmmmmmmmmmllmmmlmlmmmmmmmmmmmlmlmmllllllllllllllllllllllllllllllkllllllllmmlllmllmmmmmmmmmmmmmmnmmmmmmnnmmmmnmnnmnmnnnmmmmmmmmnnnnnnnnnnnnnnoooononooooooopooopooooooppooooooooooooonnnnnmnnnnnnnnnnnnnnnmmnnmmmmmmmmnmmmnnnnnnnnnnnnnnonnnnnnnnnnmnnnmmmnmmmmmmmnnnmnnnmnnnnnnnnnmmmmmmmmmmmmmnmmmnnnnnnooonoonnnnnnnnnonnnnnoooooooopppoooopppppppppoppoppooononnnnnnnnnnnnnnnnnnnnmnonnmnnnnnonnnnooonnnnnnnnnmmmmnmmmmmnnnmmmmmmnnmmmmmmmmlllmmmmlllmmlmlmlllmmmmmllllllmllllllkkllkklkllllllllllmmmmmmlllllmlllmmmllmllmlmmmmmmmmmmlmnmmnnnnnnmmmmmmmmmmmmnnmnnnoonnooooooonooooopooooooppoopppoopppooppopppopppoopoopoooooooooooooooooooooooooooponnoooonoonnnoonnnnnmmnnmmmnmmmmnmmnmmmmmllmmllmlllllllmmlllmlmmmllmlmmmmmmmmmmmmmmnnmmmmlmmlmmlllmmmlmmmmmnnmmmmmmnmmnnoooooonoooonoonooonnnnooonnnooonnnnnooonnnnnoooonnopoooooonnoonnoopoooooooooonnnnnnnnnoonnnnnnnnnnnnnnnnnnmmmnnnnmmmnmmmmmmnnnnnnmmmmmmmnmmmmmlllmllmllmmmmmmmmmmmmnmnnmmmmmnnmnnnnmmnm

In [7]:
def split_audio_bytes(audio_bytes, sample_rate, sample_width):
    bytes_per_2second = sample_rate * sample_width *2
    chunks = [audio_bytes[i:i + bytes_per_2second] for i in range(0, len(audio_bytes), bytes_per_2second)]
    return chunks

audio_bytes=b'oooooppooooonooonooonnooonnoonnoonoooooonnooonnoonnnnnnnnmnmlmmmmmmmmmmmmmllmmmlmlmmmmmmmmmmmlmlmmllllllllllllllllllllllllllllllkllllllllmmlllmllmmmmmmmmmmmmmmnmmmmmmnnmmmmnmnnmnmnnnmmmmmmmmnnnnnnnnnnnnnnoooononooooooopooopooooooppooooooooooooonnnnnmnnnnnnnnnnnnnnnmmnnmmmmmmmmnmmmnnnnnnnnnnnnnnonnnnnnnnnnmnnnmmmnmmmmmmmnnnmnnnmnnnnnnnnnmmmmmmmmmmmmmnmmmnnnnnnooonoonnnnnnnnnonnnnnoooooooopppoooopppppppppoppoppooononnnnnnnnnnnnnnnnnnnnmnonnmnnnnnonnnnooonnnnnnnnnmmmmnmmmmmnnnmmmmmmnnmmmmmmmmlllmmmmlllmmlmlmlllmmmmmllllllmllllllkkllkklkllllllllllmmmmmmlllllmlllmmmllmllmlmmmmmmmmmmlmnmmnnnnnnmmmmmmmmmmmmnnmnnnoonnooooooonooooopooooooppoopppoopppooppopppopppoopoopoooooooooooooooooooooooooooponnoooonoonnnoonnnnnmmnnmmmnmmmmnmmnmmmmmllmmllmlllllllmmlllmlmmmllmlmmmmmmmmmmmmmmnnmmmmlmmlmmlllmmmlmmmmmnnmmmmmmnmmnnoooooonoooonoonooonnnnooonnnooonnnnnooonnnnnoooonnopoooooonnoonnoopoooooooooonnnnnnnnnoonnnnnnnnnnnnnnnnnnmmmnnnnmmmnmmmmmmnnnnnnmmmmmmmnmmmmmlllmllmllmmmmmmmmmmmmnmnnmmmmmnnmnnnnmmnmmmmmnmmmmmmmnmmnnmnnnmnnmmlmmmmlllllmlllllmmllmllmmlllllllmmlmlmmmmmmmnmmmmnnnnonnnooooooonooooooooooooppppppooooopooooooooonnnnnnnmnnnmmnnnmnnnmmnnnnnoooonononnnnnnnnnnmnmnnnnnmnnnnnnnmnnnmmmmnnmmmmmmmmmmmmmllllllllllllllmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmnnmnnnmmmnnnnnnnnnnnnnnnnnnnmnnmmnmmmmnnnmmmmmmnnnnnnnnnnnnnnnnooonnnnnnnnooooooopoooooooppppppppppppppppppppppopppoooooooooooopoooooonnnnonnoooooooooooooooopoonnnnnonnnnnmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmnnmmmmmmmmnmmmmmmmmmllllllllllllllllllmlmmlllmlllllllllllllllllllllllkkkkkkkkkkkkkkkkkllllllllllllllllllmmmmmmmmnnnnnnnnnnnmmmmnmmmmnnmmmnmnnnnnnnnnnnoonoooooooooonnnnnnnnnnnnnnnnnmnnnnnnnnnonnnonnnnooonnnnooooooooooooooooooooooooooooooonnnnnnnnnmnnnnnnnnnnnnnnnooonnnnnnnnnnnnnnnnmnmmmmmmmmmmmmmmmnmmmmmmmmnmmmmmmmmmmmmmmmmmmnnmmmmnnmnnmmmmmmmmmnmmmmmmmmmmmmmmnnnnnnnnnnmmnnnnnnnnnnnnoonnoooooooooopoopoooooooooooooooooooooooooooonoooonnooonooonnnnonnnnnnnnnmmmmnnmmnnnnnnmnmmmmmmmmmmmlllmlmmmmnmnnnnnnnnnnnnnnnnnmnnnnnnnmmmlllllllllmlmmmmmmmmmmnnnnnnnnnnnmnnnnnmmmmmmmmmmmmmmmmmmmnmmmmmnnmmmmmmmmmmmllllllllllmmmmmmmmmnnnnnnnnmnnnmnmmmmmmmmlmmmmmlmmmmmnnnnnnnnnnnnnnnnnnmmmmmllllmmlmmmmmmmnnnonooooooonnnnnmmnmlmmmmmmmmmmmmmmnnnnoooopppppppoppppooooooooooooooooooooooooooooooonononnonnnnnmnmnmmmmmmmmmmnnnnnnnnonnnnnnnnnnnmmnmmmmmmnnmnnnnnnnnnnnnonnnnnnnnnnnnnnonnmmmmmmmmlmmlllmlllmmmmmmmnnnnnnnnnnnnnnooooonooonnnnnnmmmllllllllllllllllmmlmmmmlmmlllmmmmmmmmmmmmmllllkkkkkkkklllmmmnnnooooopoooonnnmmlllllllllllmmnooooopppppoponnnnnmmmmmmlllllllllmmnoppqqqrsssssrrqqponmmlkjiihhghhhijklmoqstuvvvwvuttsqpnmkihfedbbaaabcegjmpswz}\x80\x81\x82\x82\x81\x7f}zvspkieb`^\\ZXWWVWXY]agms{\x82\x89\x8f\x92\x95\x95\x93\x90\x8b\x86\x81{vqmieb_\\YVTRQQQRSUWZ]afls{\x83\x8a\x92\x97\x9b\x9c\x9a\x97\x91\x8a\x84|vpkgdb`^\\ZXVTSQPPOQRTX]dks|\x85\x8e\x96\x9d\xa1\xa2\xa1\x9d\x97\x90\x88\x81ysmhda^[YWUTRQONMMMNORV[bkt\x7f\x8a\x93\x9c\xa2\xa5\xa5\xa2\x9b\x94\x8b\x82yqkea^[YWUTRQPNMLLLLMORW]dmw\x82\x8d\x97\x9f\xa6\xa9\xa9\xa6\xa0\x98\x8f\x85|sle`\\YVUSSRRQRQQQRRSUWZ_emv\x81\x8c\x96\x9f\xa6\xaa\xaa\xa7\xa1\x99\x90\x85{ric]YVTSQQQQQQQQPQPQRTW]ckt\x7f\x8b\x96\x9e\xa6\xaa\xac\xaa\xa4\x9d\x94\x8a\x80ume^YUQOMMMNPQSUVWXYY[\\^bhnv\x7f\x8b\x94\x9e\xa5\xab\xad\xab\xa4\x9e\x94\x88|qh^VQLLIMNOVSZ^X\\VXPJPLR^p|\x8b\x9b\xb2\xbd\xc2\xc0\xbc\xb1\x9b\x86o_QB?DJRZfopplcaWMKHOMMZ`hro\x85\x8c\x91\x92\x96\xa7\xa8\xa4\xa5\xa6\xa4\x91\x83{n_QKNJJS_cbefdXUSPNKSZ[dhqvz\x80\x91\x8f\x8e\x9e\xa7\xa9\xa9\xa2\x9d\x90\x7fp_RMIDGQ\\]]`e\\QKMOAET\\Y]islov\x81\x94\x8c\x90\xa2\xb6\xbc\xa9\xa5\xa8\x91xa]VEEHRUUabWXVRKBIMJU^ddlxmp|\x89\x93\x8d\x99\xbc\xbf\xb9\xa8\xaf\xac\x87oheS@JVVNSghSO^`QDP]WU^lvnnpz\x82|\x8c\x97\x9b\xaa\xb0\xbc\xb1\xa5\xa5\x90}o__SPMMTWTTXZRKRVPMSce[dqwicv\x86\x88\x88\x90\xb2\xb3\xaa\x9f\xa2\xad\x8bpmmgLFYZKDSZQHJUULJSa^VYpsa[q\x80zs\x86\x99\x9f\xa1\xa7\xb2\xae\xa4\x91\x89\x85saXcbQJN]ZJFQUOEMXZYYafdc^fqst|\x90\x9d\xa8\xa9\xb3\xb1\xa8\xa0\x90\x87{ja`ZWQQTWSMNPQOOVY]_acgnh_iy\x7f}\x88\x9c\xaa\xae\xae\xaf\xb1\xac\x98\x80wxn^RV\\VOMRWSNLPXVTY_k_\\djhZgx\x80\x80\x84\x97\xad\xb7\xb4\xa3\xa6\xaa\x96~gkl]QJSYSMLKXVROQ`_ZX`mg[]ckfmz\x88\x8f\x9b\xb0\xb8\xb5\xa2\xa0\xa5\x95}cak]MER[WNKOVZRRS_`\\Z\\eke_^gorw\x81\x8f\xa1\xaf\xb4\xae\xa5\xa5\xa1\x90|kde\\VQRV_ZTLV_[RQ[fd]`dnkb`aoutz\x85\x93\xa2\xa7\xab\xaa\xa7\xa4\x95\x84{kd\\WVRQVZ\\URW\\ZUS\\a_]]affgdglsz\x7f\x85\x93\xa3\xab\xa9\xa0\x9d\x99\x8f\x81ob_ZVRPU[_]YX[[XTV[`acdehhhihlqv}\x82\x8b\x96\x9c\xa2\x9e\x9a\x96\x8d\x84xkaXVVSSV[^]]\\][XY\\_`aefilmllmqswy\x80\x8a\x91\x93\x94\x96\x96\x90\x89\x81vmbZWVUUX]`bdcba`acddegjmonmoqstvw~\x84\x88\x8c\x8e\x91\x90\x8e\x8c\x86~umgd`]]`aacccdcccdddeefhhggiklmmov}\x82\x86\x8b\x8e\x91\x91\x8e\x88\x80vnga[WWYZZZ^__^_bcaabbccddehjlnpv~\x83\x87\x8b\x90\x95\x95\x93\x8c\x84|sjb[WTTUVY\\^abeghihhhgggefgikklot{\x81\x87\x8c\x91\x96\x99\x95\x8f\x86~vmf_ZYZ\\]^_adeeghhhgffffdefhjkmpx\x7f\x84\x87\x8b\x91\x97\x97\x92\x8a\x83|rhb[WVVXZ\\^_ceefefgfffeedegijlmqv}\x82\x85\x8a\x8f\x95\x96\x91\x89\x82yqhaZVVXZ\\^`defeffeddcdeeefhjkknosz\x80\x85\x8b\x8f\x94\x96\x94\x8c\x84{qhb\\WWZ]_bfhiighgeeddefhiijlmklor{\x85\x8b\x90\x94\x98\x99\x97\x8c}pg`\\ZWX^fkkjgeb]WVVY[\\^bfihhjkjjkpu}\x8a\x94\x98\x96\x93\x93\x8d\x80sfa_^^\\_gllfdda]XW]`bdejmnnlopqojosu|\x8c\x98\x9d\x96\x95\x99\x93\x86shhf_[Zbjlgace_WQV]`]^dkmiilnrolmpstq\x80\x8f\x97\x95\x89\x8f\x94\x89yedjd[V]hid]^db[SV`bc`cnpokiqrpliosqpm{\x8f\x94\x8c\x85\x8d\x95\x86qfjpeXXble[Z`c`VRY`caakokihllhmkllktxu\x84\x8e\x96\x96\x89\x90\x8e\x81zgfjaa^_jeed`c_]`\\afiomlkoqmhipqkilwyu\x84\x8f\x97\x92\x8a\x95\x94\x85zmrqbbfijafgd^Z__][]ghhikrjaegmidopqrn}\x84\x87\x8d\x88\x8f\x90\x7f{smk^`d\\^`chcbcbb^[dhghgkmgdeijdcjlpnt\x86\x89\x87\x87\x88\x97\x8bzzvym]hgge]jmhd[hg[`bomakqtm]hlfc^ptmqt\x84\x85\x80\x89\x8b\x92\x8b\x83\x82\x7f~qfkhe``lkggfli`bfmgcjrodbfjc_dmlklz\x87\x82\x7f\x80\x8b\x8d\x7fx{\x7fygdilbX^hh^Zfld`bkkb_alhZ]hlccrxuqv\x81}zw~\x80vtt|xotutl]fjb^[hj`glmmdccab\\`kjnqx|stzwupt\x7ftpry|ow|qolmi`hieccnlgnnti`jieeeqqlx\x7f|uq\x82\x7flpy\x84ic}}oir\x84q_uweX]wgQcvvc\\\x7f\x7f_^ewfXqtusoxsutiq\x80wgfzu^`z{]_ux^TvrYYexaSytkll{b`meukhyz{to\x81slstommrkgtqojwskg`liW]ovg\\}\x84cbx\x85eWt\x7fglt\x84yjp\x7fwfalxkci\x7f|diywefpxg^oyofpmr]hsrpinofoqjvtmrr{lr~ugntuk`{wk]pwr[o\x81rYh\x81lVh\x87sVd~rQm\x86q]i\x88wYr\x84l^f\x7f^hlwsaq{lna~o\\kkkfa{jcnqqZtvcclpl`jknje{mefnw]spsfkuqcmqmg^usnfs}ffqzmflnabutgnottcqrldjwvfpvgsjultqjovujkuqeksqnlt{njpwk_qxf[qykjqwldntmhh{qcqsrjgumjmorekspejpogpmnfmhmknmmnclnlcjqodppkjfrmenrpekuljnrqilulmioqmnsvlktnlbirjnqnrnrspvklrmpnovjowpooprmmqsmonoosqrroqpsptmokpnskrsolpopoolplrnjqlnmniornjrmojlkpmimqnjksngmirheqpgkprhnpnjkkljnhsmjkmnlqjjonopomoqmrmqoomoooppmpqnpjpnmqmoqpnmqommnmqonospmmsqlollmjrpmloqnmpolipqmplqolmoqnnmnromsppmpqonoqolnpmoppmnmpkmmnlkjmmmjlnlmjkjlkjjlmkhljiijmkgkljjknikjllikkijkklkkkkjkklijiklmkjimmklklmllnllknnmomolpnoknnopopppppprrorqqqrqqqsqqqqqrrrqqrqqqrpqqqpppqqppnpponppompoonnpnnnnmmnmmlnmmlmmlmmmmmmlmlmlkllmmlnmmmmmllmlklmlmlllmmlmlmmnmmmmmmmmmmmnmmnmlnmmmmnmnmmmmmnnnmnolnmmnmnnnnonnonoononnonnoonnonooooonoopoopooooooooopqpppppqppppopppppoooponnononnmmmmmmlmllllkkklklllllklkkklklkllkkllkllllllmllmlmmmmmmmmnmnnnnnonnnnnnononooooonnnonooonooooonnnnnonnnnnmnnmnnnnnnnnnnnmnnomnmnnnnnnnmnnnnnnnnnnnnonnnnnnnonnonnnnnnnmmmnnmmnmnnnnnnnmmmmnmnmmmmmnnnnnnnnnnnnnnnnnnnnnonooooooonononnnnnooonnnonnmnmnnompnql\x81xmvilajg\\hpplpxqqzsfmnidoqlntklsnnkqlijimonpuouyvpgkheeggfjnpnrurqmojmhlkhmnnmonnkmliklollslnoojkonjnmnmntlopqqioohhllihmohmpnnnpplnnnojoqoqqtrotrpmqpjkkjigimhjnkjllkkrtw||\x80}wxohc`b]bhfspqwnne_`QZ_ay}\x8f\x9f\x9a\xa8\x9c\x8d\x84i\\I<?;JWi{\x84\x97\x99\x96\x90\x84yfZTLKOX_hsxyzwke^OXX]qx\x90\x98\x9b\xa7\x9a\x93\x81odGHA@QRm|\x84\x96\x96\x99\x8d\x82yaYQIFN\\Xhrlpkideqk}\x8d\x92\xa2\xa4\xa7\x9b\x8c\x87fYS?CHN]q{\x87\x8f\x90\x87\x7fwaYSILMXadvtxwpm_^^]jt\x82\x8d\x9d\xa1\x9b\x9d\x8d|l[OCFIQak{\x85\x84\x8b\x85wrfYTQRN[a]nmfsfdk^gms\x84\x89\x99\xa2\x99\x9e\x93\x80v`UMAILVdn\x7f\x82\x82\x89}sl^VPOOM^_`rlosilgcoow\x87\x8e\x98\x9c\x9f\x9a\x89\x84q]VMFKT[cu{|\x82~ulf]QWXScjhrxsorg_e]Ykp{\x8c\x97\xa1\xa0\x9f\x9c\x87zl[PHJMR_kt~\x82\x81}unbYVSRT]eeqtounfidbqt{\x91\x92\x9f\xa4\x9a\x9d\x86zmVTDGNMeit\x86|\x83\x7fqk_YPPZS`oivzstpjabbYkqv\x8e\x91\x9a\x9e\x98\x96\x80xiQPEAKN[fq~\x7f\x81\x83uph\\VSUOYd^orl{kqndnhwz\x82\x97\x8d\xa2\x9c\x90\x97wucMU@MRRjjx\x80x\x84yku^^bYa_jiiyioxbmmahnqs\x86\x87\x89\x96\x8c\x88\x86tkbTPPKU\\_orx|yzongZcYXc^fljrpnrkmosu|\x88\x83\x8d\x96\x81\x8b\x82lkaRQVRVfgnx|v|xhqg]gf`kpirwiqqdkkdivl|\x8b}\x92\x90\x82\x8d{ql]VRSQYdgq}x~\x82turdcb][dcaolkunormws{\x85{\x8e\x8a\x7f\x8cwsl_[QZRZiavvs|rqjff\\ed^qjixlnsikihgipks|s\x80|x}pujfj\\bd\\heglnoqtrsqnkiecdadffppw\x85~\x89\x8f}\x8b}ms\\\\VSYUjcm\x80m\x82{pxige```dfjnquuwvuupqlmmnnrrtttprkik_da[b[`b`klr|\x82\x86\x8d\x8e\x89\x88\x81vph`[_X]j`puk{rmudkeaibllmwovwnvnopkonmsmrqmtmlqhmlflggiehghlkrtw\x80~\x83\x84~~umk]^]T_`aonrxuvrongjhelklurtxuvurropmnpkronsmoojnhkjhnimlkoknqkromsoqrospppmnlkhhichfciefjijoorv{z\x7f\x81z\x80zothch^bfemjqqksjhmcghejimnouptxqwvqvsrrrrnpnimihkhkjjmknnmpmpropqopoonlomjollomoooqpqqopmoomnmllkkjkkkmnnonooomonmoonopooppqpppppoonnmlmjjjhkihkijkkkklklnmnnmnmonoonpnopoopnpnmmlljjkjjkkjlllmmlmnmnmnnmmmmnlmnmmnonnonnollkkkjkkjkkkklllmmnnnnnopppoppooooooonnnmmmmmlmnmnnnooooppppooopppppppppppononnnnnnnnmnmnnnonnonomnnmnmnnmnlmnllllllmllmmmmommnmmnmmlllmllmmlmlmmmmmnnnmnnmnnnnnnmmnnmnnmnnmmmmmmnmmnnnnnnmnnnnnnnnnnnnonnnmmmmmmmmmmmmmmnnmnnnnnnnnonnnnnnnnmmnmnnnnnnnnoooonoooonnnooooononoononnnmnnmnnmmnmmmnmmmnmmnmmmmmmmmllmmmmmmmlmmlmmmmmnmmnmnnnonoonoppoopopopooonooooooonoooonoonnononnoonooooonnoonnnnnnnnnnnnnnnnnnnnnnnnmnnnmmmmmmmmlmlmllmlllllllllllllmllmmllmllllllllllmlmllmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmnmmmnnmmnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnonnnnnnnnnnnmnnnmmnnmnnmnmmmmnnnmnnnnnnnmnnnnnnnnnnnnmmnnnmnmnnnnnnnnnnnnnnmnnnnnmmmmmmmmmmmnmmmmmmmmmmmmmmnnmnnnnmnmmmmnnnnnnnnnnnnnnnnnonnooooooooooononoooooooonnnnnonnnnnnnnnnnonnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnonnnnnnnoooooonnnoonnnnnnnnnnnnmnnnnnnnnnnnmmmmnnmnnmmmmmmnmmmmmmnmnnnnnnnnnnnononnnnonnooooonononnnnnnnnnnnnoooooooooooooooooonnnnnnnnnnnnnnnnmnnnmmmmmlmlllllllllllllllllllllllllllllllllllllmlmmmmmmmmmmmmmmmllmmmmllmmmllmmmmmmmmmmlmmmmmmmmnmnmmmnnnnnnnnnnnnnnnnnonnooooooooononnnnnnnnmmmmmmmmmmmmmmmmmmmmmmmmmmmmnnnnnnnnnnoooonooooooooonooonnnnnnnnnnnmmnnnnmmmmmnnnmnnnnnnnnnonnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnmnnnnmmmmmmmmllllmmmmlmmmmmmmmnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnooonnnonnnnnmnnnnnnnmnnnmnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnonooonoooonooooonnnnnnnnonnnnnnnnnnnnnnnnmnnnnmmmmmmnnnmnmmmmnmmmmmmmlllllllllllmmmmmmmmmmmmmmnmmmmmmnnmmmnnnnnnnnnnooonnooooooooooooopooooooonnnnnnonnnooonnnnnnnnnnnnnnnnnmmmmmmmmmmmmmmmllmlllllklkkkkkkkkkkkkkklkklllllllllllllllmmmmmmmnnnmnnnnnnnnnnnnnoooooooooooooooooooooooppppppppppoooooooooooooooooooooooooooooonnnnnmmmnnmmmmmmmmmmmmlmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmnnnnnnnmmnnnmmmmmmmmmmmmmnnnnnnnnnnnnnnnnoonooooooooonnoonooooooonnnooooooooonnnnnnnnnoooooonnnnoonnnnnnmmnmnnnnnmnmmnmmmmmmnnnnnnnnnnnnmnnnnnmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmllmmmmmmmmmnnnnnnnnnmnnmnnnnnnnnnnnnnnnnnooooooooooooooooooooooooooooooonnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnmnnnnnnnnnnnnnmmmmmmmmmmmmmlmmmmmmmmmmmmmmmmmmmmmmnnmmmmnnnnnnmmmmmmmmmmmmmmmmmmmnmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmlmmllllllllllllllllllllllllllllllllmllmlmmmmmnnnnnnnnnnnooooooooppppppoppppppppppppppppppppppqpppppppppppppppoooooooooonnnnnnnnnnnnnnnnnnnnmmmmnnnnnnnnnnnmnmmmnmmmmmmmmmmllllllllllllllllllllllllllllllllllllllmmmmmmmmmnnnnnnnnnnnnnmnnnnnnmmnmnnnmnnnnnnnnnnnnnnooooooooooooooooonnnmmmmmllllllmmmnnnnoooooooopoopoooonnnmmmlllllllllmmmnnnoooopppppppooonnnmmmllkkkkkkkllmnnopqqrrrrrrrqqponmlkjihhgggggghijkmnprtuwxyyyxwvtrpnlkigfdcbaaaaabdfhknqtx|\x7f\x82\x83\x84\x84\x82\x80}xtplhea_]\\[YXWWWWXZ]afmsz\x82\x88\x8e\x91\x93\x92\x90\x8a\x84}wqkfc`^]\\[ZYXWWVWWXZ\\_ekqx\x80\x87\x8e\x93\x97\x98\x97\x93\x8d\x87\x80yslgc_][YXWVUTTTTUVWY\\_elrz\x82\x89\x90\x96\x99\x9a\x98\x94\x8e\x87\x80ysmheb`^\\ZXWUTTTSSTVWZ_fmv~\x87\x8f\x97\x9c\x9f\xa0\x9e\x99\x93\x8c\x84~xrmhd`^[XUSPOOOOOPRSV[bjs|\x86\x90\x99\xa0\xa4\xa5\xa2\x9c\x94\x8c\x84|tnhc^ZWURPMLKKLLMOPSX_gpy\x84\x8f\x9a\xa3\xa9\xac\xab\xa6\x9e\x95\x8b\x82yqjd_[XUSPMJIHIJJLNQU\\ep{\x86\x91\x9d\xa7\xad\xaf\xac\xa5\x9b\x90\x85{ske`\\ZXVTQNKIHGGHILOV`kv\x83\x8f\x9c\xa6\xb0\xb3\xb2\xaa\x9e\x8e~oe]XWY]adda\\SIA=;;=CMYi}\x92\xa5\xb3\xbb\xbc\xb9\xb0\xa2\x91}k]VVZcmtvrj`RC6,*-4?KXeq\x80\x91\xa3\xb0\xb7\xb8\xb3\xaa\xa1\x95\x89|qfabgnstpfZNA6.+.5@NYdkt\x84\x96\xa8\xb3\xb7\xb6\xae\xa5\x9c\x93\x8a}pf`adhheaXQG<3,,1<IT^djv\x86\x9b\xaf\xb9\xbc\xb5\xad\xa6\x9d\x94\x85td[Y^dhf`XPI?7.+,5CS_imx\x8a\xa1\xb6\xbf\xc0\xb7\xab\xa3\x9a\x94\x89{nbaejnicXOH<3*\'/7GQZfjy\x8a\xa3\xba\xc3\xc3\xb6\xa6\x9e\x92\x8c\x7frh`chmpiaTIB71++4>LSYdh{\x91\xaa\xbd\xc0\xbc\xb1\xa3\x9e\x91\x8a|nd^chlmd^QJA71-1:AKOZbn\x86\x9f\xb9\xc4\xc3\xbc\xad\xa3\x98\x8b\x81rh^_glpkd[NE6-\')2<GRYfk}\x92\xad\xc3\xc8\xc2\xb3\xa2\x99\x8f\x87yl`\\bltsk^OE:2,+06?JT_h{\x95\xb2\xc8\xcb\xc8\xb7\xa8\x9d\x91\x8b|pe_flrqf\\OG=3+),4<HQ]i{\x96\xb4\xc8\xcf\xc6\xb6\xa6\x9a\x91\x87yk`^fpvsh[LA8-&$\'1;FWct\x88\xa5\xc0\xcb\xc8\xb7\xa5\x98\x8f\x88\x80tkegouvl\\L?850.-,26IXj\x85\x9c\xb7\xc0\xc2\xbb\xa9\x9c\x8e\x84~uqklqstm`SC;4/--.3;Nat\x89\x9d\xb3\xbe\xc0\xbb\xa9\x9c\x8e\x87\x82|vpnstuk]K:30//-,.:Qg\x82\x9b\xb2\xc1\xc3\xc0\xb6\xaa\x9d\x8e\x84|xustuvodUE80-++()2E`z\x95\xaa\xb7\xbe\xbd\xb8\xac\x9c\x8d~wsqqstsmbTE8/*(&%&3Ie\x83\x9e\xb3\xbc\xbf\xbe\xb8\xac\x9c\x8a{roprvurj_QD90)#!!\':Sp\x8b\xa1\xb1\xb8\xbd\xbc\xb5\xa8\x95\x85xssuz}zti^PD;2*" ".F_{\x96\xac\xba\xbe\xbd\xb5\xaa\x9c\x8c~uqruz{ypdUI>6-$\x1e\x1e\';Ur\x8b\xa0\xb0\xb7\xba\xb8\xaf\xa2\x91\x81unmqvzytkaWMD:.&#(6Jb|\x92\xa1\xa9\xac\xaf\xac\xa4\x96\x86yojjlnonljgbZOB8207AO_o|\x89\x94\x9d\xa1\xa2\x9e\x97\x8d\x86\x80~~{yvsnic[OB6.-09EUcp{\x87\x92\x9b\xa1\xa2\x9f\x9b\x96\x92\x8d\x89\x85\x81|wqjd\\RG<535:CLU]gs\x80\x8c\x96\x9c\x9e\x9f\x9e\x9a\x94\x8d\x86\x80{yurmdZPF?:9:?FNV^gq|\x88\x90\x95\x97\x97\x96\x94\x92\x8e\x8a\x85\x80{wsmf\\RKGDEGKOSX`is|\x83\x88\x8b\x8d\x8c\x8b\x89\x86\x83\x81\x7f~|yuog^VPMKKLMPU\\env{\x7f\x81\x82\x82\x82\x82\x83\x84\x85\x87\x87\x85\x82}ulc[UQPQTVZ]bhnsvyz{}\x7f\x81\x83\x83\x83\x81\x7f|xrkd]YXXZ]_acdgilnoqstvxxxwutsroljgeccbcdeeghjmnprstuvvvusrqqpnlkihhhhhhiklmopqqrqrrqpooppppoonmllkjihijklmnpqqponnmmmmmnopqqponmmlkjjkmopqqqqqqqponnoopqqqrqqqonmlllllllmmnnmkkjjjkklmnpqssrpnmkkjiijklmmmmmlkkkklmnopqrrrqonnmlllmmmmmmlllkkkkklllmnnnoooopppqpppponnmllllkkklmnoopppppppooonnnmlklmmmnoqrrsz\x85}h]hulXSd|\x80uloywlfc`^frm^^t\x86~kix\x89~[MbxnURk}xqry}wojjkjeeghiijkllmquwtpsyzsmpvunjmqngcfiheegilllnoprqnopoopprttrrqonmljhgikjfdgnolkqxxqotxvqnqusnkmpnkkklkihhhiihikorqrtssrpqrpnopqonppomjijigfgiihinommoqqqrssuuuurqojhfccb]bg`bghihippquv{\x7f~\x7f\x81\x81~|wtojfa^]XZ[[^^`eeglnry}\x82\x88\x8c\x8f\x90\x8e\x8d\x87~wmb]VROPQPUWY_^^glotz\x84\x90\x92\x94\x9a\x9b\x98\x90\x86\x80uiaZTRQOQUTVY]][\\cims~\x89\x93\x97\x9c\x9d\x9e\x9a\x8c\x84~og^VURRQORSRRUWUZckqx\x85\x93\x98\x9c\x9e\x9f\x9c\x90\x85zqdWTSNLMSXQRXWXWS\\gen}\x89\x98\x9a\x9c\xa7\xa1\x99\x94\x7fxsbZUXVM[YRYSXXMTW`gfy\x86\x90\x9d\x9b\xa2\xa7\x9d\x99\x8d\x80zmfa[YW[URUPRPISUQ_fk~\x82\x8d\x9d\x98\xa2\xa4\x97\x9c\x8f}\x81seh^]]TUTLONEHNOVbit~\x88\x98\x9a\x9d\xa6\x9f\x9b\x99\x87\x82|mhd\\\\`TOTLLK>FNGQ\\gx|\x8c\x9c\x9d\xa5\xa7\x9c\x98\x91\x80wpd`^\\\\YXWPNJ@>?@DQbn|\x8b\xa2\xb0\xaf\xb4\xb5\xad\xa3\x91\x7fuk]VXX[YV\\XOMA=A<@P^q\x81\x92\xb0\xb9\xb7\xc0\xba\xac\x9e\x86wiZSOT\\^_ce_QG=5/-9Pdx\x91\xac\xcb\xd3\xcc\xcc\xbc\xa8\x8elZK><=L\\enpnk^H:/+.6Nj\x85\xa2\xbe\xd7\xdc\xd6\xc9\xaf\x98xVE82;ARenupjbJ:+$)-B_~\x9f\xb5\xd0\xe2\xda\xca\xb4\x97|ZB52:DRfu~|pfT>0\x1f\x1d(8Zv\x94\xb7\xd0\xe8\xed\xd6\xbf\xa6\x84fC.-0>MZq\x80\x81zjYN8&$%7Vq\x94\xb4\xce\xe8\xee\xdf\xc7\xa6\x86hD-%):N[n}\x84\x83oXI7*# .Ll\x8e\xae\xc5\xdf\xed\xe7\xce\xac\x8coO6&&5IYfs}}pW?/"\x1d +?b\x8a\xaf\xcc\xdc\xe6\xe9\xd9\xbc\x97rXC517F]ox{vkY?\'\x13\x0f\x19(;Wz\xa9\xce\xdf\xe2\xdb\xd3\xc2\x9ez\\JFFJWi~\x88\x86ziT; \t\x00\x0c 9Yx\xa1\xc7\xdc\xe1\xd6\xc4\xaf\x93x`NLR^n{\x87\x8e\x8c\x81hN7 \x0f\x04\x0e"=^}\x9f\xc0\xd1\xd8\xcd\xb9\xa4\x8araSRZgx\x82\x8a\x8d\x88~hN7#\x14\x0b\x12(?a\x80\x9d\xbb\xcd\xd2\xcb\xb6\xa0\x87o_ROU`p{\x83\x84}rbM8$\x17\x11\x14\'@[z\x94\xaf\xc3\xca\xc6\xb5\x9f\x89qaWQU]kz\x83\x87\x82vhUA-\x1c\x13\x0f\x1b6Ss\x8d\xa5\xbe\xce\xd4\xcc\xb6\x9d\x84obXUXao~\x87\x88~p_K8\'\x17\x11\x10\x1c8Wy\x96\xac\xc1\xcf\xd5\xd0\xba\x9f\x83i[URW]iy\x84\x8a\x85vcM9)\x1b\x12\x0e\x15-Nr\x93\xaa\xbf\xcf\xd5\xd5\xc6\xad\x91r^TQV[ds\x7f\x89\x8b\x81qXC1!\x19\x12\x11 ;_\x85\xa1\xb6\xc5\xcc\xd1\xc9\xb7\x9d}bQJMS[gs\x80\x88\x87~kS>+\x1f\x17\x13\x17)Ek\x8e\xaa\xbe\xca\xcf\xcf\xc2\xad\x90pWGAELWes\x80\x88\x87|iR>-"\x19\x14\x1e4Sx\x98\xb3\xc7\xd0\xd5\xd0\xc0\xaa\x8akSD?CJXfu\x82\x89\x87}iTA0%\x1d\x1a&:Z~\x9c\xb6\xc8\xd0\xd3\xcb\xba\xa1\x80aJ=;@IWgw\x86\x8e\x8d\x82oZG7+""-Ca\x82\xa0\xba\xca\xd2\xd2\xc8\xb7\x9d|`H;8;FUcs\x80\x87\x88~lYF90()2C_}\x9a\xb3\xc0\xc7\xc8\xbe\xb0\x97x]E759DR`nz\x80\x83|o_MA834:G]v\x8f\xa5\xb3\xbb\xbd\xb6\xaa\x97\x7fgSD??FP[gry{wncWME@@FO]o\x81\x92\x9f\xa6\xaa\xa9\xa4\x99\x89yi]VTVZ_ekmonic\\VSPPQU[epz\x82\x88\x8c\x8f\x8f\x8c\x85}unkhhhijlllkieb^\\ZYZ\\_chmsw}\x80\x83\x84\x83\x82\x80~}{yvusrpoligdb`^]]]_acgjnqtwz{|||zywtqomkjhfdddca_^^_acehjnsx{~\x80\x81\x81\x81\x80~|yuronkigfedccccdefhjlortvxxyyxwutqomkjiggffddddddeegijmoqrttttttsrrqqpnmljjihgfefghjlmnprsttuvvwwwwwwwvtrnligfeccceghjklnopqqqrtuuvvuutrpnkhfdcbbbcegikmoqrsuvwwxxxxxvuspnljigfeeeefghiijkmnpqstuvwxwvusroljhfeddcccdefghijlmoprstuvwvvutrqnmkkihggghhijkkmnoopqqrssttttttsrqonlkiihghiijjklmmnopoppqqrqrrqooonmomntwqmnliea[[`efimpvxz{{vrqrqlijmppoprttspnlhffeccegjlnnnmlkkjhhjnqstuvvtrolkjigghiijigihhhgijlnnrw{}~\x81\x83\x83\x81|ytnhca`^^\\]^^^[ZZ[_ajx\x89\x99\xa4\xab\xb2\xb0\xa6\x90vbN?3/9HZlx\x82\x87\x82xk]SGABEOYj\x85\x9a\xab\xb2\xb7\xc2\xbe\xab\x93wfXG=9AR_jr{\x81}qcZVOHFLW^chp\x81\x8f\x99\x9a\x9d\xa9\xab\xa3\x91}siYJBGQY_foxuj_ZVOHHMTX^bl|\x89\x93\x96\x9b\xaa\xad\xa3\x93\x81xlXJDHUY`it~xjb\\YPGKSX\\^gq|\x8d\x93\x99\xa1\xac\xb6\xab\x9c\x8b~t^LEGQV[eq}{nc\\[SFGO^^]hou~\x85\x92\x9a\x9e\xac\xaa\xa1\x92\x81weQEEMTZfu\x80\x7fvke[NFENVZ^fmqv\x84\x8d\x91\x96\x9f\xad\xa7\x96\x8a\x81r`KHLPU\\ky|uphcRIGIPTVahljq\x83\x8a\x8f\x91\xa2\xae\xa5\x96\x88\x83s]KKQTUbq{{spi\\NBDIJPXbnnly\x88\x8b\x91\x93\xa6\xac\x9d\x93\x89\x81q[RTVWZgtyuqleXJEJNOSZimjlz\x8a\x8a\x8a\x95\xa8\xa9\x9a\x8d\x87\x82nXOWYW\\iwztpmfWHGKMNSaiinp~\x8b\x89\x90\x99\xa7\xae\x9b\x93\x90\x84t]U[XW\\gvwrqogYKHJNJP]gjlox\x84\x85\x88\x8e\x9f\xa4\x98\x8e\x8b\x85q^UVWRXcmtqnpf\\OIKKKQZcknqz\x8c\x8c\x8c\x95\xa1\xa9\x99\x8d\x90\x85q_T\\XOXeqqovuj]TTQKOWbdfrwv\x82\x8a\x93\x98\x9a\xa6\x9f\x95\x90\x80seWVRTZajprwre`WRKIQV]besuu\x84\x89\x8e\x94\x96\xa3\x9e\x91\x8f\x82udWTPNPZbllsulg^XSMRUU`eirt\x84\x8c\x8c\x97\x9e\xa1\x9e\x90\x8c\x80m`UPNHP]_hprulid[TMSUPXainp\x85\x8f\x8d\x96\x9e\xa7\x9f\x8f\x8d\x83paQSPIP[binuyspic]TVVTY^gno~\x89\x8f\x96\x98\xa5\xa2\x93\x91\x86thWSPGOX^clxysqoj]RVUPQWagjx\x8a\x8e\x94\x9c\xa5\xa6\x99\x92\x8byjZONEFOW`enwvsplgZVVUTS[fhr\x81\x8a\x94\x97\x9f\xa7\x9d\x96\x90\x84ueWSJDHOT[cmrpttnhca`[\\acfks}\x81\x86\x8d\x94\x93\x8d\x8a\x88|pf`^SRYXZ_bifgmkiiinjhjjkiltuv{\x81\x89\x86\x84\x8a\x86~xqoj`^_\\]__ccfjhknmpljlgfhgjjkrvy{~\x82\x80{xtpieb``aadefjjjkjjieggcccdgfjquz~\x81\x85\x85\x82\x81|wsolhfggfggjihjjjhghgeeeefgjnrw|~\x82\x83\x83\x82~{xuqnmkihhggfhhhiiijjihghijlnrvz~\x81\x83\x84\x83\x80}{wtromlkihgfeeeeedefffffghhikoquxz{{zyxvsqonmjihfdccddeefghhhhhhghhjlmqtuxyz{zyxvurpnligedddefgijkllljiihhhilnprsuwwwwvutrpnnkigfffffghhiijjijjjkjlmmnpqrsuvvwwwvsqqnljjiihiijijkkjiklllmmnonoppqqstvvwvutroljigfffghhijjjjkklmmopqqrtuuvvvwvvusqomkihhgghijjkkllkllmnnoqrrstuvvvwwusroligedc`bdeefgjkkjkmlmoqstvxxyzyxvusqnljhfdbaaaacdggiklnmmoqtuy~\x7f\x82\x84\x81\x81\x7fytnhd`_`_bdhiikjjheca_bcho{\x8a\x8f\x95\x9d\x9f\x9d\x8e}tfYNKSXZepuusqme]\\]^adlnlnnljfv\x85\x83\x8e\x95\xa1\xa4\x8a{vgXGCQSR_hrskmlc`]dgelqnhefa]`i\x80\x8d\x91\x9f\xa7\xa9\x96zpdPGCNVWfroolnhY\\d_`fryjjpg[V^kn\x85\xa1\xa0\xa6\xa9\xa4\x9eta`MGCG__aonqlce_`gdhnsyoikh^VYhp}\x9d\xad\xab\xa7\xaa\xa9\x88hb]QFGY_Ycjidad_[ceiloungnh[Z]hid\x7f\x9e\xa0\xa3\xa2\xa9\xa5whhUICI[TRdia\\\\h`Udnjcnysfik^UXdjj\x81\xac\xae\xa0\xae\xab\xa6\x7f^jaBCR[\\ScreZdid[jvtnu\x81vgiocMVhedk\x93\xaf\x9d\xa7\xae\xb0\x96jljSEI]bUfsngbkhYoqlsluk_ihTWYZeht\x8a\xad\xb1\xaa\xa6\xa6\xa4w_h\\JHV`\\Wpu_`ll_`qvljlr`VbdXTWg`cu}\x9b\xa8\x99\xa7\xa2\x9c\x89dliQKZcYUioaYcmcZjqkfjriZ`a\\STbbbqs\x8c\xa3\x9e\xa3\xa0\xa6\x95oqjcUTgdTikb_^fg_iuopsxq_ajZKZb[^n~z\x90\xae\xa8\x96\xa3\xa2\x89genYIWfZXikc`hhZdskmmop_ZeaVUYebau}\x8b\xa9\xad\xa6\xa8\xa1\xa0{ekcPK[`WYmm[aid\\aqnmrsmhfeaX\\[[ccpuw\x99\x9f\x94\x9d\x99\x9f\x80atiML\\bRUkiZ[eeZ_ljdmsh`gj^[\\b`Xjrn\x7f\x9f\xa0\x94\x9e\xa4\xa1zlzcQR\\dVZmiY\\eaX`qji|\x80uspvjdf]dgdmxv\x8b\x9f\x9a\x9d\xa2\xa4\x95wxr^YSZ_Scd\\ha[`]jfgwrpplolgf]ch`fus\x87\x99\x91\x9c\x9e\x9b\x9ayzt\\]RS[P`c[d`_[Xdabkmrpprpmgabcdblnq\x8b\x92\x91\x98\x9d\x99\x85|vh[TTYRYe^`g`\\^ca_innptsrmla^h_`jjm\x81\x90\x92\x95\x9d\xa5\x91\x80\x83x`YZYTWedcjic`ddceloopsurmjfdcbfkn\x80\x90\x8e\x97\x9f\x9d\x95\x84\x82s`_VSXWcdcjgd`ae]ciinnpwolmgec`ffel\x7f\x83\x87\x96\x93\x93\x8e\x82zmfYR\\WXachcfgacb_ddihiplimfbcb^`fhw\x80\x85\x91\x96\x98\x93\x8a\x83ukbWZYZbgkmmmjgebcdgjjlrplllgdefehv\x7f\x84\x91\x99\x99\x99\x95\x8b\x7fvk`^_]bhmnoqnihfb`cedglokmqlijigirvz\x85\x8c\x90\x92\x90\x8e\x83yrga^]`achikjhieceeegimmlmmkihfdcgjpx\x7f\x86\x8a\x8a\x89\x85}ungcbbbcdeeeedbcdefgfhigfgfdcddgkpw\x80\x86\x8b\x8e\x8d\x8b\x85}tlfcabdghjkkljjjjjkjkkjiijhhhilnrx~\x85\x8a\x8d\x8e\x8d\x88\x83zqkfbbcdghhijihgffgfdecbbbdeegikpuz\x80\x83\x86\x87\x84\x81{tnhdcbbdefggiihiiiiffebbbdfgilnptw{|~~}|xvrnljhgfecbbbdeghhihghffghklnopqqstuuvuvvuutrpolkjfdeefhikkkkjkkkmnqssstutsrqpopqqrrrsrqqolkiiiijjkkllkllllmoqqrqrqqpommnnopqrssstrpnmlmlkkkllllllklmmmmmmmnnnnkkklllllmmmmmkjjjkkkklklllmmkkkkmmlmnonnnnnnnnooppopponmnnmllmmnnnnnnopppqqrrqpononnnooopqqrqpqponnnmmmnnmmmmmnoonnoonnnoonopponnnnooooopppppponnnnnmmnnnnoonnmmmmmlllllkllkkklllkkllkllkkkjkkllllmmmmmmmmmnnmmnnnnnnnnonnmnonnnnnnnnmmnnmmmnnnnnooooonnnonnpommnoonnoonmoommoomllopoopoooononmhdozvc`n|{pdi\x8f\x97f2F\x8e\x9f\\\x19>\x93\xb6\x87GR\x85\x93tWUgrnbo\x84{darxdXcrtmks{~\x81tb[cm`NQj}xik{\x84v`Zhwqdap\x7f~umjiklhcagv|ytt{|pdcknebkvyphhntmcds\x7f}rkpz{sgdltpgcjt{xpkp|zm`drtmeju|vkglpleelrsmnqssmklkihggfgllhhovwtmggkkd\\^istqoqwytmgefiklopuyyumjkomgfkopqrqmghnokfgosrpoprsutpmmqtoigkojdehmkhijkkjopqssxzzyvxxtqqpokjlljfdddb````acghmy\x83\x89\x8b\x8d\x91\x92\x8b~maZUSOQXdr|~|{yukaYTRRUY\\_diry\x81\x8c\x92\x96\x98\x96\x93\x88udVNKGHO\\ly\x82\x86\x86\x82zog`ZXZ_eimomkhejns{\x81\x8e\x95\x94\x91\x84viYME?DM[m{\x86\x8e\x8e\x8c\x85ynaXVV[afloqqnnqv|\x82\x89\x91\x97\x97\x93\x89zk[QJFINZjv\x80\x88\x85\x83{qk^UPNV]dloppnmhejs\x81\x8c\x91\x96\x98\x95\x8czfSE?=?IXj{\x89\x91\x92\x8c\x85{pcXRRX`gnqsttsomqx\x82\x8a\x90\x92\x90\x8a\x7fq`PD>>BM[iy\x84\x8c\x8c\x88\x82yqf[TSY`eklllmljjov\x80\x88\x90\x96\x97\x92\x89}o`QEAAGQ]kx\x82\x88\x89\x86\x83}vnfb`adehhghhkmlnry\x82\x87\x8a\x8a\x87\x83}si]UOLPU[dkqvxxxvspnllmmllkifeeeegjnruy|}|zwtpjgdcddfikmnoqrstttsrqnlifedcddeegjlmnopqqppponnmmmmmmmmnoppoopppoooonooponmmooqqrssttsrqonmmlkkkklmnooooooonnmmlllmlkkkklmnnnoooonnmlkkkjkkkkjklmnnoppqqrrrqpoonmlkkkkllmmmmmmllmmmmmmmmlllllllmnopqqqqpponnmkkkjjjjjkjklmmnnoooonnnmllmlmnopqrsssssrrqppooooooooonnnnnnnmnmmmnmmmmmmmnnnnoooooooonnmmmmmmmmmmlllllmmnnonnnnnnmmmmmmnooonnnnmllllllmmmmmmmmllllllmmmmnmmllllklllmmmnnnnmmnnnnooooooonnnonoooopppoonnnnmmnnnnooopoooonnnnnoooooonnnnmnnnnnnnnmmmmmmlmmmmmmmmmmmmmmmmmmmmmnnnnnnnmmmmnmnmmnmmmmmmmmmmnnmmnnnmmnnmmmmmmmnnnnnnnnnoonononnnnnnmmmnnnmnnnnnoooonoonnnnnnnooonooonnooonnnonnnnnnmmmmmnnnnnnnnnnmmmmmlmmmmmmmnmms\x87\x8bnP[\x80\x84Y2H\x81\x99|Y`\x84\x90uZ[gaYv\x98\x8bXFs\x9a\x81?"M\x8b\x9brGY\x94\xb1\x88H;g\x8e\x80K;j\x9f\xa2sSe\x81\x86gLWt\x86yfgu{se_ahosolszwj^dsvjbn\x84\x84k^k|v^Yixuhhw{rikroiimnkltvngipqgagqqhgrysgeoulcfrwldkxyldjvyodeqynacrzk]fwzj[i~zg\\o\x84x`^s\x83v_bt\x7fs_atyn_drsmgimmnnjiknphhmospnqqsrpnsuslhmpha_hoicbjmjfkoqppy~}\x81\x86\x89\x86zwxpe`dhd_blohegkcZ]dihekssrt\x82\x87\x83\x7f\x84\x91\x88uruqcT^gc^bjld`fggeaikiienkfefjnt\x82\x86\x88\x93\x96\x8e}ooiXQ\\diaiz{ofekaV\\golgqyoa`cgdn\x86\x91\x8c\x8f\x99\x9f\x8angj\\KOaojlz~sh`e_SU_imluxqh``][am\x84\x95\x93\x97\x9a\x9d\x98wce]UMSkpnvxytd__\\[Y`lqqqrsh^^_cdf\x80\x95\x94\x93\x96\xa1\x94qbg`OM[hnjq{ylb``YRWaffioqne_a_]ck\x82\x97\x90\x96\xa3\xa0\x96yfjZHLRbiew~ssmdcYZ^\\fprrqvqa]b_^]h\x8a\x91\x8d\x9e\xa5\xa7\x92vvkSILU\\^l~zvxuh\\X\\WP\\jmnnwvd_a^\\[l\x8e\x8c\x91\xa8\xab\xa8\x93{zgHAEHPRj\x80uy\x84{n`]`PN\\gijt\x7ftfji_[]h|\x86\x92\xa4\xa8\xa9\xa5\x8d\x80rVIA=AFYkku\x87\x82vnigTKTYZ_guulpoha^ds{\x82\x92\xa2\xa7\xa4\x9b\x92\x87k[R?B??Z\\d}{\x81\x83yzoae]Zd]ingvlitfgis~\x7f\x89\x9a\x9f\x98\x97\x95\x85veWSB>NOTfmw}x\x80zpocgh\\ligrkhohfdkwmw\x88\x85\x95\x89\x89\x98{tvZ[RDVVOemfwqr\x7fgosftgkzbsqeseen`tjj~v}\x88y\x8a\x83v\x80njj[hcWldbpeqqispovgvvdrjhlcbp\\dqdzpv\x8a{~\x8a|~xptjcgf`jbnhnnlpjokllnhlijigigj_rpgypuyk{phsghgjgkflngllnilljnlgtiinjjkiuhqtn\x81iu\x7fhvnokmjqjmrqlrrtpvqotgu`kiajcifkinnkukxpttqoqofugonlspourpuprnqmomqhshsdrhnjllnjwgzkqknelbijflokqmqkpjngmjmknpnnlokjmikikmnlqolnkjkjklmrppstmqjqillmmoppoqnpmnpknnnqqounqnmlojonppqsrqroqnonmnlpmnmonmnnmmplpoqoonknimimjnomrmrppnqmonnmompkrkoloknlpnnornqoolkmljllonmqqmonnkjnikjllkknilinjimlnkorhrmlkolnkrnnqpponqlmomnnnnmnmnlnnlmnmqjslnonoiojnmmppoopsokzmjtnkhqncnrjirvgpwmlqrijrljonokrqiqojojonhpkllkokjolkjmminnmknqinolnhonfpnlmlshmnkijknjmripqpmntmmprlpprmltmnlqkhnpgmkmkhojkiknlhrpmoqrmpsnoprqspvpopmnilohinhllnnjtolqpnoqtlrvpunpnljgikhjokkihjdcioox|\x86\x87\x81\x84|ulccYWa`firwmqqhdW_YUZaz~\x7f\x9a\x9e\x9e\x91\x86\x88hRSFKDPljw\x80\x81\x89omkRWOQ]Zdebvji\x85\x83\x85\x8f\x96\xa9\x8e}\x8fpZRL[KRpsv{\x7f\x80icjXQXaf_eteahklr\x8c\x98\x91\xa0\xa9\x9f~tvWCM\\]Zs\x81~zy|mXYYYU]qn`abgVX\x84\x91\x8d\x9a\xad\xbc\x9f{\x83kMDD^U\\z\x82\x82vsyeXQU_RZnkn\\`iS[l\x88\x98\x88\xac\xb9\xa8\x97p\x82g;LN_]R~\x81hgjr[D\\\\RS]pcXb]b]\\r\x8f\x94\x91\xa7\xb6\xa3\x81vx[FLZ^ZgxxejoaPVaXOhtg\\fmYRfe|\x91\x9c\xad\xa8\xad\xa3~wjWQTea`r\x7fjaqlXR^dPYrldah`L_fw\x99\x98\xac\xb2\xab\xaf\x84uoTWIVh\\irrk^kgTY^]W`kfac]T[et\x96\xa7\xaf\xb3\xb0\xb9\x97lmaUKKgg]lri]]b[K[aZ^fqfYd[[a^\x81\x98\x9b\xaf\xa0\xb2\xacprgUQE]f\\hthd_f\\NV[\\Xfugde_^Zc\x84\x9b\xaa\xb3\xb1\xc0\xb0\x82ph^HD`gfcq|fVebSLUg_]nsn\\Yda\\f\x8c\xaa\xa5\xab\xb0\xbf\x98nmaUEHefakpp`ZbYPRY^bhsncaa\\aj\x8a\xa6\xa7\xb2\xb3\xb7\xa8togQLJ^l^gzm``c_OQ_`\\`pv`[cdb^q\x99\x9e\xa1\xaa\xae\xb4\x86mo_QESkd`lwmY[aXJK\\aXcrlbW`k_i\x8d\xa6\xa7\xa4\xad\xb9\x9dphfXIF`oggqyn]^_XTQZfflffia`bg}\x91\x9b\xa7\xa9\xb7\xab\x83zn_RHYeeinxvc`]WNJU\\^lkgf_e^Zt\x90\x96\x9a\xa7\xb9\xb5\x8duteO=Haa^kz}jbh^OKOYZ^kmjcafgf\x80\x96\x9b\xa5\xab\xb6\xab\x84xlZN@N]]gox|ohd\\UONW_dfhjhecek}\x91\x96\x9e\xab\xaf\xa4\x87vn[IAMYV`ozyplmbUQU\\^_flmgfjkr\x82\x8a\x93\x9d\xa2\xa9\x9b\x89}paNHOQUZevuopnl_WZ\\a]^iifhglko~\x82\x86\x91\x95\x95\x8d\x80|q_TQRTQZgklmorjbdefc`hkhffljgpw{\x81\x84\x8f\x91\x88\x85\x80xn_^[W]`chkpploonlkmqnjjlidccgioz\x81\x84\x8c\x8f\x8d\x89\x80{vjda`cbdhgklklklmlmmnnjggd``dhjnw\x80\x84\x84\x84\x88\x85zsmhd`_aadgfhjjjiijkihjifdccabefhouz}\x7f\x85\x84\x81{trmgd`bcacefihilkkjjlighgfcbfeeknx~\x80\x87\x89\x88\x85~zwlfbbc``fhijkonknommlmlhgfdghhmrx\x80\x82\x88\x8e\x8e\x8b\x86\x81{qjda]\\^abeiknnoppponmkiffefhilqw~\x82\x87\x8c\x8f\x8c\x87\x82|ri`[XTVX\\adhonoqpolifba^]`afijsz}\x87\x8a\x8e\x91\x8c\x88\x82xne\\TPRRU\\bhmpsstqmkhd_\\``]dlory\x84\x88\x8a\x91\x95\x95\x92\x8b\x88\x7fog^SOLLQU_gjuywyvsoee`Z\\[^bfmpr|\x80\x84\x8b\x90\x95\x95\x93\x91\x89~seXPGBEIOZcltz||xtofc]Z]Y]dbkmp~~\x82\x91\x92\x98\x9a\x96\x97\x8a}t^RK?=BEN[gnw\x7f~}ytmfc\\Z\\X]_aihr~z\x8a\x96\x92\x9f\xa1\x9a\x99\x89\x81pYWH;EBGX_ltz\x83}}|rmfd^Z_\\_ddkjq}z\x86\x91\x93\x9d\x9e\x9d\x9d\x8e\x83u`VH=??CQZgqx\x81\x7f\x7f|vphe_\\[Y\\[^dalus\x84\x8d\x92\x9f\x9f\xa0\x9f\x8e\x85t]TD8;9AOUgqw\x83\x81\x81\x80wsjeb[[ZZ\\\\b_gtt\x81\x8f\x96\xa1\xa3\xa4\xa4\x94\x89zcUH=:=CN[iu~\x86\x88\x87\x85\x7fvqjb_\\ZXX\\]_fnr~\x89\x93\x9d\xa2\xa6\xa2\x9a\x8f~jYK>89>GUbp{\x85\x89\x8a\x87\x82{qic[WUSUVY^cmt{\x88\x91\x98\xa0\xa1\xa1\x9d\x90\x85vbUH?>=BOZft}\x86\x8b\x88\x88\x83zsj`]UPRPRXZaio{\x85\x89\x96\x9a\x98\x9a\x93\x89~oaUIEBBJS[fpx\x80\x82\x81\x81}xslid`]\\[[\\^dhlsy~\x86\x8a\x8c\x8e\x8b\x88\x83zrkd_\\Z]_cfkpsuwxxxurolhdba_^^_`cgiouw|~\x7f\x80~zvrojgfeddffghhiijjkmonponmljiijjlmqrtvwwwvusrqponmlklkjihhhgghijjklmlmmlllllmlllkklkklnnonoonnnmlkkklmmopqssuuttsrqqqqqpponnnopqrtuvutrpolkjkkmmnopooonlkiihijklmmmlkjihiklmprsrqponljiijkmnqrstssrpnkiiiiijjkkllllmmnopqqqpnnljjiijklnoqrrrrpnlkjiihihiijkllmoppqrrrqponmlkkklmopqrstttsqqonnnmllklllllmnoppppppqoonmmmmmmmnnnoooooooooonmllkjjkkllmnnoonooonnnmmmmnmmmnnnnoonnpomoomkkkjjkjlmmnmnonmmnnkmmkjlmkjlmlmonmnpnpoolonknonnoprqoppsonpqppnntrmlptmjmrpompwtjelulahqtkfpvojhnpjinpspkpomidlokhhnuqhktqgagrnfgqvmaivqeamuiaix|kentq`\\mrhfly|kjpsrfgrupgjsusee{uillvsenplrhhjktpkmmjicq\x7fnprpocl|oupo\x83qp|~\x87ugwpjbctk`fhij`loi^\\kriom|{imn}\x8fzt\x82\x86\x8eog\x80\x80z\\PoofXNs{_THctZXYx\x82]Ym\x81xbWs\xa4\x89qk\x82\xabxOaw\x95j?c\x83\x7fV<cs\\RNemZ]\\ch[_girwow{\x83\x8a|\x80\x87\x8b\x8dympxvh[mwq]WnkZXUkmagjgf^grbamwzz\x88\xa4\x93{x\x83\xa0\x88il\x80{bWm\x87|[bhjUIevf^VamSOhuu[b\x7ftsr\x86\xa0\x7fu}\x91\x9en[o~mO[\x85\x82`R_n^FQilYO]paHNoy^]o\x8c~_m\x88\x8c\x86t\x8b\xa5\x83uXr\x95a\\l~\x80ILjlbGQn`VT`sWLfhefoznu\x80v\x83\x90\x89\x93\x8e\x88\x84u}oesmqfbo`Y`Z^_U^c_bedj]]tmddn\x8d\x8b\x90\x9c\x87\x95\x95\x81\x8fw~\x86ckqrz\\_plhRQi_[^cr_VUVhjbepvwkx\x8d\x8c\x92\x8a\x97\x94v{nolchuh_\\^`XLX_bbX`c`ZTaec]]jw{\x84\x8c\x9d\x95\x88\x90\x8f\x8exqztcfrekhou^W]bh`iun\\Z^igX\\mrst|\x8d\xa0\x92\x90\x97\x9f\x9a{bw\x88nabwxYTttaMXpkY[juZFXqnTRjton\x8e\xad\x9f\x82\x86\x95\xa1z]s\x80mSYzvUXnq\\M]rcV`uiQP`iWOgwifq\x83\x92\x89\x87\xa0\x93\x8b}jtlgdftb^mkh_Y`\\XY_fe[Z__\\Vafgr{\x94\xa4\x92\x8d\x90\x96\x8didrvi_hvm\\dmhaT`i]gnmk^Zc_Z^ftsk\x81\x95\x8c\x90\x98\x9d\x8buqwpdfpug[kqhVXieWTdrm^\\b[XMYsty\x9a\xa0\x9a\x91\x86\x98\x84bhdhi]iqcf`]f][_Yei_bfhXNZ__f|\x89\x90\x9a\x96\x9f\x9d\x8a\x84smk^elljefgid]_da^^jmka[]ZT[isx\x8b\xa0\xa4\xa0\x97\x9a\x8foieb_[kqihfgd^_a_^Yemmkb^_]bj|\x97\xa9\xa8\xa6\xa6\xa5\x95rid_YXhlkigmf\\\\Y][V]gkfY]a^`fs\x87\x96\x9b\x9f\xa3\xa5\x9a{lneWP_ne]hsnWQ^_TJVmm_[em\\Vgv\x85\x8b\x8f\xa0\xa5\xa3\x96zso[UOYgadouq]S^YJGXg]Rbh[]gs\x80\x8a\x98\x9f\xa2\xa5\x9c\x8bzmcZVZ]^gmplfhcWOR\\\\Y[bg``ov|\x89\x91\x9f\xa2\x9f\x9f\x95\x86zkc`[]_aknmomibTUWUSS[]T^ox\x84\x8f\x9a\xa5\xa0\xa0\x9c\x88}nc_TV[\\bejlhd_]XSTUVX[aafrw~\x8c\x92\x99\x9b\x9a\x9c\x8d\x7fzld[V][]gipoknhb_Z[WU\\\\]fkv\x84\x90\x98\x9b\xa0\xa5\xa0\x91\x83{o`TRWWW`imnmnmfb`^^[\\abglnu\x7f\x83\x87\x8c\x91\x96\x8e\x87\x82~te_`\\XY`ffeiookjnrmhimlgdisuuy\x82\x8a\x89\x84\x83\x82|qiggcadhjhhlnmkkmligffd_`defjottuvutommljikljggjkjilnmmkkkfcehjjlptvwxyywuutstssqonomjhiiihhiiighjjklnpqssttrqrrqqqpnllljihhiijijkkkjjkkkkmopqsssssttttsqnmmkigfghiijkkiikllkklllllmmmmopqrqqomlkifefgghhiiijjjklllmmnnmmnopqrrrrqpqomkjjkjkkllkklllklnooooppoppqqrqqqponmlkjkklmnnmnmnnmnoopqqppqrrsttttsrqoonmmnnooppooonmmmnnopoopppppppppponmlljjjjkjkllkkkkkllllllmllllmmnnnnnnnllkkkkkkklllllllmmnnoooponnnonoopopoonmmmmmmmmmmnnnnmnooppppopppnmnnnnnonnnnmllmlmnnmopnjz\x83pahulXYksmhr\x7f~spvukefifdglnppoopoljmnkjpsrpqrpoonmnoljmomhilmkkntwoipupgejmlihlrqlloqnknpomorronppkiklljkmlmmnnnoollopnmlpqpoorqooqppoonkkkkjhgikjijkmnnoswvvx{zurqqmhefhgegihgghe``cfgglw~\x87\x8a\x88\x8b\x8c\x85zkcbZTSYbehijonjd`fhfglonjjov~\x88\x8c\x8d\x91\x91\x8b{h`]XRRZgnmnqrnd^bfhhly~wnhid[Yav\x88\x90\x96\x99\x9f\x9b\x83jZUSNMVerupoqpgYYbjmjs\x80\x7ftc^`]Z]q\x90\x9f\x9c\x98\x9b\x9e\x86aNQSOKZoyuqpoh^Z_gnrsx~|sd[YSR\\g|\x95\xa2\xa3\x99\x94\x91xXIOUQO^qwnkjjd]_hnrnqvuj`ae]NUjtz\x8b\xa6\xb2\xa2\x8d\x90\x8bsSI[eYScstf_lpfaftvllstj]_f_UZjrnp\x84\x99\x9f\x99\x94\x99\x96}eahiZVelh^YhnbZ]lqdcpzr_Zhl\\S[ruij\x8a\xae\xa0\x84\x8e\x9e\x9alG`ueMPrw[S`oiTXlqifq\x7fzkghf[S^igcly~\x87\x93\x9b\x9b\x8f\x85}m_VU[\\[acffdddaadflnmookc^cb[]dnnlz\x8f\xa1\x9f\x94\x99\x9c\x8ctbefWT_gf`emk`djhihouopupb`fc[]hmilr\x7f\x92\x93\x90\x97\x9b\x90ziigZV[eb_ehjgcgg`cijnnpndcd^`\\^efpp|\x9c\x90\x8d\x9e\x98\x9bz`}kU][mjUnrbgffd\\hnlsyzsgmo_X_he_lxv\x87\x93\x92\x97\x95\x93\x89rlj_\\ZYbcbe`gma]chjbjzunklqa\\ecbbgpn{\x92\x8e\x8c\x95\x98\x9a~q~o]]\\efVgrfgliichkhovspqqkdaca\\bgfgq\x83\x83\x88\x8e\x92\x91{zvh[S]]OZiifmwpfnngejrmksrlkheb`acehw\x84\x82\x8b\x93\x90\x90\x83}|d^^RUQXhagvtuprrdfhbgeilfmieiffghjltx}\x80\x82\x85\x81yyoic^^Y\\bafmpnqwtquvtuttuqqqmqpnsomnpnmpqlmnmlkmmkmonnoqnmqpqqqqponjkkhijjkkkkjjijkllllllkkklmkmnmlmnnnprrrrsrponnmlmnooppoonnnnnnmmmmnmmnmlllkjjjijlmmnnonlkkjhhijjklllllmnoooooppoqrqqqrqoonnmnoopqqrpponmmmmnoppqqppppponmmmmmmmnnnnnnmmllllmmnnnnmmmlllmnnooooonnnnnmmmmnnnopppqqpooonnnnnooonnmllkllkllllllllllklllllmnnopppppponnnnnmnnoonnnmmnmmnnnmmmmmllmmllllmmnnnoooonnnmmmmmmmnnmnmmmmmmnnnnnnmnmnnnmmmmmmmmmnnnoooooooooooooonnnnmnnnmnnonooonononnnonnnnlmlnkmlnjonqmw\x8f`pndZRo]Uv|op\x84\x91`r\x84galri^vzbopqidnm]jmggmprgytmqprllqqnoqqrpqsqnqqomqoknnmklkflgfkhhgjkikmjoqqrpsopqnmlnniqpmonokikegeegdffhjdovos~\x81}{\x84{nif]VV\\aektzzzwrmfcbcfelonnmmkghv}\x80\x8a\x93\x94\x8a||jRPPQOZrw{\x85\x8b\x86yqqbX[_efnxuusqogfd^b`dfy\x8f\x86\x91\xa1\x9c\x91wznJKOSUVu\x81{~\x84\x84tcfdYYcmst{{wpedbZ^cggfsqn\x7f\x87\x8e\x8e\x8b\x95\x89rndXONWW]nrtz|wmhjc^`prnuxwg_e_Z]cfbilgt\x86\x87\x86\x8a\x94\x93{moeOJQ[[\\p~wv}{oehibcnsusvwibc`]\\dgemomqz\x89\x8c\x87\x98\x95\x88|nn`QUY^`gtvsurngfeeglrtsuuldcfc\\cggdekr\x80\x86\x87\x91\x99\x93\x81tufTTW^[_ssmqtpc_ie_dtzrnvweZdlbWcqg^iow\x7f\x84\x8e\x8e\x8e\x90\x80sk`ZVS\\`cjmopjigefegnrsrqtocahh__hmgdlpv\x7f\x88\x8e\x91\x92\x8f\x82vpf]VZb`dmstmkpmedinonswtohee``_bffdijo|\x83\x87\x89\x8c\x90\x84vljaUQX_`cirsmknmieiprprttoffgecbgjijnr{\x82\x85\x89\x8c\x90\x8e\x80ysibWW][agmuuuutrlikkklmpnjhgdb_befgjox\x81\x82\x86\x8a\x8a\x88|qlbYQOUX\\bkuyxwyxpjknlijooljjkiefilmns{\x81\x80\x81\x82\x84\x81xpnke^]_``behjjjlnnmnqqooomljjklnoprqppnonnnnoonoononnoonnnpooprttstutrponljkkmnnnoonlmmllkllllkllkjjjjhhhiiikllllmmlkjjjijklnnpqqqpppppppooonnoooooponnmmmmnooppqpponnlllmnnopqqppoonmmlkllllllmmlllkkkkklllmnoonoonnnoppqrrrqqqppooonnnnnooopppooonmmmmmmmmmmmllkjjjkkkllmmmmmlllkllllmnoooopoonnmmmmmnoooooonnmmmmmmmnnnnnnnnnnmmmmnnnooooooonnmmmmnooopooooonnnnnnooooooononnnooppooponmmnmlmnnmnnnmmmmmmmmrsmprsmiimhhgoymnvslgcia]kfensslpvmglmhbhoggmnrmnwtpqpupkrtsuryzqqmmja`ecdbkokmrpolmmjlmmssvyy{xwvronlijlmjnrqnomlhdedbbdglkkqqrrrvwuyz|yqsqga`ca]afkjhkonjhlmjghmokox\x81\x82|\x83\x8a\x85xoojYPV][Yeuwqszxibghc^hqompuoedgc`cjx}\x82\x8c\x92\x92\x8a~uj`UQSX]hqw~\x82\x7fxqnia_bikmpu}qhjhf`av\x84\x81\x8c\x96\x9e\x98\x81yu_QNJTW\\syt\x7f}wobedV]cjslszqiefaZ_gkv\x80\x8f\x9a\x91\x96\x91\x84w_ZTJIO[gnt|\x80}sjfe[W_hllpzwnieeYW`bm}\x88\x93\x96\x9b\x9d\x8czneTBFOUZh}\x81\x7f\x81\x81wic`YV[dimsyvqng`]Z]cs\x7f\x87\x91\x9a\x9f\x97\x83\x7fo[PCIML]jsz{\x82|pld_XV_aelswsrogd__afw}\x85\x92\x96\x9e\x91\x84\x82naULQOVaju{\x7f\x7fxtnd_^aeemtvwsrnfd_baew|\x85\x8f\x93\x9b\x8f\x85\x82oaUORNSaksz\x81\x83}uqla\\^adfnwvttqlb`cakrx\x85\x8b\x91\x94\x88\x86|i`RLMGP^cnw}\x80yxshb`_``fmoqrspjjhfmqu~\x80\x88\x8c\x83\x84~rj\\YRKQT[`grsuwutmmmglilpjqpnqmonkrrtxy}|xywqmkiecdgeeilijnqqpswuttssonnllklklnnmmlkjhhhijkkllnlkkkjjknnprrrqoomklknoorrrrrrqoonmmmnmnoopnnnmmmmoonoppnmkihgfghijllmmmmmlkkkllmnoopqqpoonllmmmnopqqqqqoonmmmnmnnopppponnmmmmoppqqqqpooonmmnnoooonmmlllllllnmmmmlkkkkklmnoopponmllkkkkkmmnnoonnnmmlllllllmmnmmmmlllmnooppqqpoonnmllmmnnnnnnnnnnmmmmmmmmmmmmmnnnoopppppoonnnmmmnnnoooooooooooooooooooonnmmmmmmnnnnonnnnnnnnmmnmmmmmmmnnmnnnnnmmmmmmmmmmmmmnmmmmmmmmnnnmmmmmmmmmmmmmmmmmmmmmnnnmnnnnmmnnnnnmmmmmnmnnnnnoooonnnonnnnnnnnmmmnnnnoooopopoqnrzmpligegciqorutsqpmkmmno~ooucgdd\\`pgmxuxuyultlhmhiinlmpkoplnmklmlkonmnqmnqnnqoonrkmohpijqgolloiqljrklphqjlqiqnoonsnoslmnljhqhjskpqqtorumrqnomqnlunkskjjghbgk`nnhsoqrrxpvzqzxsytsqlpijlfjjehheecga`heiux~\x88\x8e\x90\x8a\x8f\x87rsfOPJEHM\\]l~y\x88\x8d\x82\x86}sm`]RMTJQ`]m~\x82\x90\x99\x9e\x9a\x93\x93|lfMFB=AEV`i|\x82\x89\x90\x8d\x8e\x86\x80|mgaWTNRPSfdt\x89\x89\x9b\xa4\xa2\xa5\x9b\x91\x7fkbG?@3>KRfs\x80\x8d\x8f\x96\x91\x89\x84tl_RSHELKTZetw\x8c\x97\x97\xa8\xa6\x9e\x9d\x8e~k]L=<8<GRbo~\x88\x8c\x94\x8c\x87\x82sm^UUFKMHUYfs{\x93\x98\xa3\xb2\xab\xad\xa4\x91\x85iWJ5447FRes\x7f\x90\x8f\x93\x93\x85\x83tgaRPJCMIO\\`p{\x87\x9a\xa0\xa9\xaf\xa7\xa1\x95\x81lYI8114=L\\lz\x87\x8f\x90\x8f\x89\x80uh]TKGEEKOWfq}\x8e\x9b\xa5\xac\xad\xac\xa2\x92\x83nXJ;557BR_q~\x8a\x93\x92\x92\x8d\x83xlaYPLKKOUZdpy\x86\x8f\x99\xa3\xa2\xa2\x9f\x93\x87weYKA@?EOZgs{\x84\x87\x87\x85\x80zsle`\\ZWWX[_fls|\x84\x8d\x92\x95\x96\x93\x8e\x83ync[TQRTY]dhlprtvvxxwusqlgd`\\[Z[_bhmtz~\x81\x83\x82\x80~zvqnjgedccdegjlnppponjheb_]\\]_bglsy\x7f\x83\x86\x87\x84\x82~xtplifeeeffghjjllllkjihgfeeffhjlosuxz{{zywtroligeedeefghiiikkkkkkjjjjklmoqstvxyyzywvtsqolkihhgggghhhhhhiiijjkkkllmnoqrsuvvwwvwuutrrqpnmlkkkkklnnoopoonnmmmmnpqrsttttssrqqqpnnnljjhhgggghhiijjjjjjjjkllmnnoooppppoppoonmlkjihhgghiijkllmnnmnnoopqqrrsssssssrsrqpoonllkjjjkklllmmnnnnnopqqqqpqppqpqrrrrrrqqpoonlmlllllllkkjkklnoprrqqqponnnnnnnonommmlkkkkihhhgghihhjllnonoqnmnnnooorrpooomljklkjkmnljllkjjlmnmnqrpprrrpqrrrqrronmkjkklllmjiiiihiknoppsvustvyxsvzytmnojebgifbcklebgkf`ekoliry{yr{\x81}wrz\x82vkgkl`Z`ee``ehgc`dgeaafjklls}\x82\x80z\x85\x8c\x84xmxzoceuqc`cjaU\\bda^kqhbclh[^jrpl}\x8f\x8e\x85\x81\x91\x97\x81pk{|c]iwt[\\jkbRXolcckyq``il`R_vyqv\x92\xa3\x93\x7f\x85\x9d\x95t]q\x84sWXuz\\UjwiSWmp__jviUVik^Tbupkr\x81\x96\x9d\x92\x8e\x91\x9b\x94uirzpZWjnaWdpfQRbj_XfqiVObnbT]txhh{\x8d\x8c\x83\x8e\x9c\xa0\x8aswzugZkq_\\\\hiZX\\`dafkkiaX_`]X\\ijio|\x83\x85\x8d\x96\x97\x99\x92\x87}sulcciiaZ]deaXZdgfbhqnfYXgeXUasnhu\x87\x9c\xa3\x9a\x98\x9a\x9b\x91|nnvob]jpaR]kn^Xftm^`lm[NXkgYXkud`o\x80\x8b\x90\x96\xa0\x94\x8d\x90\x83rcfqjXYgj\\S]icUUej_YajeSPafYUeqj`n\x82\x83\x83\x90\x9b\x9f\x90\x8a\x8c\x7fofmufchhcX]jaUValcZispaP^pbQXiuf[v\x8a\x82\x85\x8f\x9e\xa9\x8f\x82\x88\x7fuhcmpie_^ca^][chabfovfTS`o`R`ooc_|\x8f\x8e\x98\x9d\xa2\xa7\x95\x8a\x87wtkbmlih`fh_YXahhffkme^\\`hjecdjnihv\x7f\x85\x91\x9a\xa0\xa2\x96\x86|skc]adgaZ^b^VSZhiccgli^\\]dk_Xbnrjdu\x83\x83\x8d\x98\x9e\xa5\x94\x84\x7frld[cgggbcid][\\cjfejllg_`fga]`hmjjsv~\x8c\x9a\xa4\x9f\x94\x8b\x83tdZ[aabcehhca`acfilmmmlgeeec]^giilqx\x7f\x8d\xa0\xa8\xa0\x96\x92\x8d{d[`db]aktqhfgd_^dmqnkigda]\\Z[^achmr\x80\x92\x9e\xa0\x9a\x95\x93\x8aubXWWUU[epsnjhhgc^bimligikf\\WY^`_bjq|\x8b\x99\xa1\xa1\x9c\x98\x8e\x80o`URQRV_ktxwtsqjd__bdefhklga][\\\\]_fu\x86\x95\x9c\x9d\x9f\x9f\x99\x8d}k\\SLKMS]gov{|zunifdfhknrvtpljhd`^]belu|\x80\x83\x85\x89\x88\x82yrlhd``bdfggghhgikmprsvxvrmjhd`]\\^^^`beimqvy{|}|yvtsrrqponnmlkjkmmoqrpnkkjgda`___adiou{\x7f\x80\x80\x7f}{wsponnmlkjhffhhggfhjkkkkmmljgeefghjnsx|\x7f\x80\x7f~|{yxvvutrpnmkjjjklllkkkjiijkjhfdcaabehlpsvwwwwwwwvvuspnljhfffhjkllmlkijjkllkjihgfgilnpsvxyyz{{{zywuqokhffgijlmonnnmkjiijkjjjjiijjjklmprsuwyz{zyxvsomjhhhijkmonooonlkkkkklllkjihggghjmoqtwyz{{yvspmjgedefgijlllllljjkjkjijiijjiijkmprux{}~}|yvspnkjhhiijklmmmnmlkjjijiiiiihhhhjlnqtw{~\x7f~|yuojea`^^adgkopqrroljhec`_^]\\\\^aelt|\x81\x86\x8b\x8d\x8d\x88\x81xne^YWX[_ekrvyyxuspmjggggijklllkjigefjpw~\x84\x88\x8c\x8c\x8a\x85{qg^XUUX^emu{}}zvrlgc`_adgjkllkjifefkt~\x87\x8c\x90\x92\x90\x8c\x83vhZPLLPW^gpz\x81\x85\x83~wpje`][\\_ejnonmlkheb_^bku\x7f\x87\x8c\x8f\x91\x8e\x87zk\\PIGIMU_kw\x80\x86\x86\x82|tmf`\\[\\aflqssrolhc``cjs~\x87\x8f\x93\x94\x91\x8a}n_RJFFLUan{\x85\x8c\x8d\x8a\x84{sjc^]^bgmsvxxvsniecflu~\x87\x90\x97\x9a\x99\x91\x84sbSICBEMYgt\x80\x86\x87\x85\x80ypg^XVX[agloqsqmhc`_`fny\x82\x8b\x92\x95\x92\x8a}m^PGA?DN[iw\x82\x88\x8b\x89\x83{qf^XVWZahntwxvrmgcadjr{\x85\x8f\x96\x98\x93\x8a|l\\ND>?FQ_nz\x86\x8c\x8e\x8c\x85}sjb]\\]bhnsvy{wrkfbaelt\x7f\x89\x94\x9a\x99\x93\x87xiXKB?CLXft\x81\x8b\x90\x8f\x89\x80vkaYUUX^djqvywrmieabhpy\x81\x8b\x93\x95\x91\x89|oaRHBDIQ]kx\x82\x88\x89\x88\x82yoe_YWX[afkptuspmjfccipx\x7f\x87\x90\x93\x8f\x87~rcTJFEGOZhu}\x86\x8a\x8b\x87}uog^YY\\`ciptvvssoiggmqs|\x84\x8e\x90\x8b\x8a\x84{p`YVPPU[emu|}\x80\x80zwrnmhhhjpnrtqulkm`daYfdguq\x80\x85\x81\x89\x81|zlgbZZYY\\_dhjmoqqqssqqoopommlljfgdacbeiipty\x81\x80\x83\x84\x7f|xplfb`]\\^`bdfjmoqtvwvvttrnkhfd```^a`bhipuy\x80\x83\x84\x86\x82\x7f{rle`]ZZ\\^bgiostxxyywvtrrponmlkjjhihhmlntu}\x7f\x7f\x84\x81~\x7fvomea_[\\^_cfhmortuuutspqlijbeb`c^ac]cedmnt{z\x82\x82\x80\x81{tphb]YZY\\`eiouxz}}}zyusqlnighgfedgdenjlts{~~\x84~~~rni`]XWXY^cgmswz||{xuqmjeecccdgeijhhkjjoqtx}\x80\x80\x83\x84~zxoifb__bdflqty{|}{yvrqljiffgdffdeehhjqpv~~\x80\x84\x83~zvogc_ZZ[]bglruy{{{wurojhebcb`ebggnpozx{\x84\x7f\x86\x83\x82\x84yxtgic\\^\\]bdimpxw{{z{vqpkee`]]Z\\[^^egitv{\x86\x85\x89\x8c\x85\x83|rk_YSMOOR\\`fptx~||{xsnkea`[[\\Y]``fnnx\x81\x81\x8e\x92\x90\x97\x8f\x8a\x83wmaZRLONS\\ajrw}\x80\x83~\x80}vsphdb^X^YWaZ`ddlny|\x84\x8d\x8c\x93\x8f\x8b\x87yreZVJMKMWZdkqyz~\x7f|}wspjeb^Z[XX]\\bnpy\x88\x88\x95\x9e\x99\xa1\x9b\x91\x8c|obYNJNHT[]mnu}{\x81\x7f}~ywtohfbY^YT`[\\jjm\x82\x7f\x86\x98\x90\x96\x9c\x8c\x8c\x84tkbWQPMOWYaimrvxyyyutpkhd_ZZWV[[_llx\x87\x84\x98\x9c\x98\xa5\x9a\x93\x91}rhYQLIGNTXcimtxx|}yzwqnhc[\\VRWUV`diy~\x84\x95\x93\x97\x9f\x90\x92\x88vtaYUHNJNVWafjqrvwvyurshhcZZVRUU[dgw\x7f\x85\x9a\x97\x9d\xa7\x97\x99\x92{wgWUHIHHUQ]icuus\x81y}\x7fuymieW\\RNYNYf_x\x7f}\x9a\x93\x96\xa7\x92\x94\x90xufYSKLFNTRcdhwsz\x7f{\x80|ywnld[[TOVPYday}\x82\x9e\x94\xa0\xa9\x97\x9c\x91}wfYQJHEKPS_fiwy|\x84\x83\x84\x81~{mmdT[RHWNRd_q}\x80\x94\x94\x9a\xa1\x96\x97\x8c~ue]QKLGMUVaikv{{\x81\x82\x7f\x7f{wojeZYUMUPVd_t}}\x97\x92\x98\xa1\x93\x99\x8a~|bbYIUJMXU_ghruz}~\x80\x80z{ukma]\\VYZWgbmzw\x87\x8a\x87\x92\x89\x89\x85ywlddY\\]Zbdeklnpqsrssrppmjieba_^^`dfjosw{~~\x7f~zytolifffdefgfijjmmnrqqqronnkkkijijjhkijmlpqrvuxxvwutsppmmkiiighihijjkljjjihhggghhikklooqttvvwyvwvtspnliifegfgijlmnoopoonnmmmmmnnooooppppqprrrrrrpqqnnmlkkkjkjklkoslslmjgjahdhgjojsqsrzrtrnmjmgijritutuvxntqjojmlipnmrnslrnonpkxeythwmrmplpfrhnkojoomjuhpmqkoolfljbgjjcnolksugsune{gojnoensglrlnonvfuuksptmlrnhqjmilpglsfqnlpf{gazf`ohkfn}lt\x84wvwwncgeS\\_[_dlhluoornoiqkhsiloklhojgpootvxuvyopohgei`ah_`c\\[^_dj\x80\x86\x87\xa3\x9b\x98\x98\x80~[MP3<HNdl\x87\x92\x8f\x9e\x91\x85|feSLXR]lm}~}\x84rnlV[SOi_r\x8f\x82\x9d\x97\x91\x97u{jO[LPWYuqz\x8e\x82\x86\x7fur]c`U`cgnn}uoyibdYe_s\x85z\x9f\xa0\x8f\xa0\x89}hS`@;^UZu~\x81\x7f\x7f\x84j`iYUaeoou\x83vspjcX_cZfolsyz{\x7f\x84\x84\x82\x82}zphg^ZVZ^_dlvrpypimedchngmunjkjg`hh_gkgkmx}w\x89\x8f\x7f\x8c\x7fzk`hTR[Ybbiunrtqqjnjhnoqnqsfejcgchmbemhgq\x84}|\x94\x86\x83\x89vuh^eSVe`elrvpxvktkjnhpohrndhbhe_okgphpkbryt\x8b\x88\x85\x99~{|miW`aVblnsszslpfgiikjlnklfgkbekdlahr`n\x7fsz\x88\x83\x86\x84{vqkeb_kf]usirklfbpgkvlopir_esafioq`svbmlw\x80k\x91\x92s\x93\x81uwdm]Wsd_ytkoqkbgkeiqomolcdghaftegukirq|ss\x8ft|\x90l\x80rbwVcqWpmbpemgaoelljvillafb`^_nfhrmpmjzot\x8bw\x85\x85w\x80mok]gaceesffqgimcjiiolljifZ``aikrmpwlt~zx\x80\x82z}\x82ntqfm^uk`xjcmhjdltfpripeiiXoc]vmnpuwevsowx\x84v\x82\x89vuvwdgvjfpojeoodmpjnoqkjleiefgglnropvgmrkzt{\x87u\x8d\x82p\x85qojbvebwgnomqeno`ongnljgdlecqjfnlkknsqr{vx|{v{rss`ok[sfdvcokcr^in\\umfveik_jjblifmjnwpxzt\x7fy{\x7fuysrmjpfhndmjjmfoiiqflldmdgg`kbgmfmjkjjolwvy\x81\x81||\x83ppw]ih]mfkrltjoodmjelihlkijhigekegnitwt\x84{\x83\x87w\x87trx_pj[udhvhqnkncoedrhmonjhmdiickckhflhpsw|~\x86}\x7f\x85ptsahc_helqqrrulmmcgidenghqihpdegbbcgikxw\x80\x84\x80\x86{uudbaW_^cjjxtu}popdffacfjforkrnijcg`bjfoyx\x83\x85\x84\x83\x80xmlaX`Y\\gfpvx}|ywpnffg^eicnpktpjogefemhw|v\x90\x81\x84\x8dw|ofeW`Z]jftxx\x80}z{qokghbiherjmwjqpghgb`meo}w\x87\x86\x86\x87\x7f}oleZ`Y^fepuw}{zwrpghe`hegnjpokqfjhcodrwr\x89~\x86\x8b|\x83vnj_`W]`^llqxwzyxtqpliihfjjhnmlqmlljfdhbfnityw\x7f\x7f{zunic]ZYWZ^bhmptwwuusnmlfhhdhhfihghgiijootyx\x7f\x7f|}ysplgddabedfiiknnpqrrrqqonnlmmlnnmonnooonoonnmllkkkjkkijjiiiiijllmoopqqrqqqpqpqqqsssttvuvwvvvuutssrqrqqqpoommmlkkkklllmlmlkkjjjjkkkllllllllmmmmlmmllllkkkkjjjijjjkkkmllmmmmmlllklkkkllllklllkkklkklllllllllkkkklllmmmnnnoooooooooononnnmmmmmmmmmmmmlllklkklllllllmmmnnnooooooooooppppqqqqppppppppppppppoopppppqqqqqrrrrqrrrrrrrrrrrrrqrrqqqpqppppppooonnnmnnmmmnmmmmmllllllllllllllllmmmmmmmmnmnnnnnnnnnnoonooonnnmmmmmmmmmmmllllllkkllllllllllllllllllmlllmmmmmmmmmmmmllllllllklkkkkkkkkkkklllllmmnnnnooooopppqpppppppppppooooonnmmmmlllllmlllmmmmnnnnnnooooooooooooooononnmmllkkkkjkjkkllllmlnnmonprf\x8abm\x83M\x84dY|Zwll~d\x82km~_vkgqdtgoqexjkvgrnmtlssovoqqnqmqomrmqposnsnvjlw`sgdpbmjdrg{lxybz[jd\\k\\ijgtkuorqnpioljmmnrlpmnlolmojqllpknlllmlmnnnrloqmookmmjmjlknknplqnoooooonnnmmnompnopnpkplmojnmmpkolpnopmqopnrnqpproqoonmojnoiqknoksmqrptornnmikgfhfhihljmmlplonlqkopnpppoqnnohmggifhhgijgnlmunxvv{vzwurplghbddaifkonsrrsookhicfhknvy{\x86\x80\x83\x84yypjbb]\\aaflnrtwuvwrstnrokpjllklknknnkohmhhngsqu~y\x83}\x7f|rwhfh\\c`cfgoksupxsqtmljfhbegajhcnfimkqmvtt\x80u}}synlibd^bcciinopsqstqqrpmolilhgjihkmjopntrtvvxvwvrsmkjdfccccffhjjnmnonnkligiffhhjkpmrsuvuzuvzrvspqnnllliljikijiikkkmmnnomnmlmmmnonooonpmonkmlkllkmmlmllkkjijiijikkknlnooqpqqqqqrqrrrsrrrrsrrrqrqqqqqpqpppoononnonooopoppppooooonoonnnmmlkkkkkkkkkkkklkkkkkkkkkkkllllmlmmmmmmmlllmmmmmmmmmmllllllllmmmmnnonooooooooooonooooooooooononnnmlmmllmlmmmmnnnnnnonooooppppppproqtlrmpnooporqqrrpqopnnnmlnlnlnnonompmonnnmnnlnmnmmnmnmkojojokmnlnnlpiqjmmjmjiofrgnmjqjqnlqmnmkojolkniollpgsgpknmolpipmmqjqlomnornlnnnpnplpmqlskoojrnlvgsojrjnknjmqmmlwcvhnjmomowjptplpkkemmqd}dskvenmq^vndukmhpiwZ\x83_qpipflpenlmngmp_~aplnrbxkcut^{ktbtzXuw_rnllhzaox_ygqod\x81[vrpjn}c_\x86Zkovrc{\x83V\x83~dti~\\l\x80fa\x84]zd|ogm\x82Wk\x88]kp{]ur\x8aF\x8erg]\x94`duyjO\x8fvRy\x80agq~gX\x88uD\x88riV\x7f\x80Ra\x96a]t\x85bXm\x995u\x88mK\x91iqawpalnlV\x87Rz~Ny\x99:w\x85ba^\x8aoA\x8f\x84Od\x88rKj\x8f_V~\x8dLp\x94pO\x81\x89Q^}t[co\x99Hh\x99wJ\x82\x87[V\x80\x80IW\xa1aG\x91\x83[f\x83~bM\x92uLm\x81r[j\x8bvPw}mY|ydjsx_]\x81|Jt{n]o{r_u\x7fj[~uQgzcarveo~jfzt\\ns`efntZl\x86o[y|oUl\x83jCy\x83eUy\x89dT\x80\x82SdyzbX\x7f}[\\\x87mabxnleuzqkuouo^su]anwk`~\x84ag\x83w^Z\x80}Q[\x84{[_\x81\x82\\e{ye]uwdfttlksnijmdkxhgu}igv\x82aavzd`u\x80kcs\x85o`o}n\\g{l`kuqfftsegtpmesypjvwsiotndekrhaovogjwtfmtrnmrutoqxxpnrvpejollefopihntpelupjhsvlnyxxtxyssolmiggdgfdhgdfgiiinmnrqrwvyxv{|vsnlmi``bcb`addc``acfegv~\x81\x80\x89\x95\x93\x8a}{~qb[^eb_emtngff_XRTZZ]hv\x83\x89\x8c\x97\x9b\x98\x8f\x80{sfZW[`_gluvtlc_YKDDMSZfy\x8a\x96\x9a\xa0\xa4\xa0\x94\x83uj]TQSZalty{xtj]RHECCITbs\x80\x8e\x9e\xa5\xa5\xa4\x9c\x91\x7fncXSPRZiqwy}|reWLF@<?HTfu\x87\x97\xa3\xaa\xad\xa6\x9d\x8ezi[QIGLXdmv{\x80{sf\\OE;9<EN]n\x7f\x91\x9f\xa6\xab\xa7\xa1\x97\x84teYSNPW`jsy}{vncWLD@AFN[n\x80\x91\xa1\xab\xb3\xb3\xaf\xa5\x95\x83qbXQNQXbjsyyvnf[NE?=>DO\\l}\x8e\x9d\xa7\xac\xac\xa8\x9e\x90\x7fl]QJGIOXakpttohaWNGDCGMVcr\x83\x93\x9e\xa9\xad\xaf\xab\xa2\x94\x84rbUMIIOW`iquvtoh_VPKIJNU^gt\x81\x8e\x9a\xa1\xa6\xa9\xa6\x9e\x93\x82rbVMHHNT^hqwyxvpja[TPOPSW^jw\x83\x8f\x9a\xa3\xaa\xab\xa8\xa0\x94\x83scXPLLSZenvz{zvpg_WRPOQTY_ht\x80\x8b\x95\x9c\xa1\xa3\xa2\x9b\x91\x82rdYRNMQV_gotvurmhb]WTQQQSV[cnx\x83\x8d\x96\x9d\xa2\xa2\x9e\x95\x88zk`WRPQV]emsvwurlf`[VSRSTWZ^dmw\x82\x8c\x95\x9b\xa1\xa2\xa0\x98\x8e\x7fqcYRONQV]elqssqnjea]XVUUWZ]bjs}\x88\x91\x99\x9f\xa2\xa1\x9c\x93\x86xi]SNKLNSZaglopoomkigedcbaabbcgmu\x7f\x88\x90\x97\x9b\x9c\x9a\x94\x89}n`SJECDIPYbjqvxywusokhedbaabcglrx\x7f\x85\x8c\x8f\x92\x90\x8d\x87}rg\\SNLMQV]elrwyzyxvtqoljhgedba`acgmsy\x80\x87\x8b\x8d\x8d\x88\x81yoe]WUTW[aipuy{{zwtpligfeefeeegjnsw{\x7f\x83\x85\x86\x84\x80ypg`\\YYZ]bhotx{{zwuspmjhfffgggfecaaabehmtz\x80\x86\x88\x89\x86\x81{tlfa]]^afkptvxxwtqmiea`__`diov|\x82\x86\x88\x87\x84\x7fxpga\\ZZ]`dinruvvuspnmlllkllmmmlkihfedddfhjnrvz|\x7f\x7f\x7f}{wsokhfefgiloqrsrqnjgda``ciow}\x82\x86\x88\x86\x83|tkb[XWY\\`fkpsttroljhhghijkmmmlljjiiiiiklnpqrrrrrrqqppppppppponnlljihhggfggijlnpqsuuuvvuutsrpponmljiiihhhhhijjjkkllmnoppqstuuvwxwvtrqommkkkllllllkjiiiiijklmmnnooppppqqrrrqqqonljihhijklmoopoonnmllllmmmmmmmlkjjjjjjklmnnnoppqqrrsssrrqponnmlllmmmllmmmmnnooppqqqrrrqrqqpppppponnnmlkkkkkklmmopqqqqrrrrqqqppooonnnnononnoooooonnmmlkjjjiiiiiiiijjkkllmmnnooooooooonnnonnnnmllkkkkjjjjkkkkkllmmmnnoopooonnnmmmmmlllllllllllllllllmmnnnnoopppqqqqqqqpppooooooononnnnmmmmmmmmmmmmmmlmmmmmmmnmmmmmmmmnmnnnnnnnnnnonnnmmlllllkkkkkkllllmmmmmnnnnooooonnmmmmmmmmmmmmmnnnnnnnnmmmmnmmmnnnnnoonooooonnoooonnoooooooooooonnnnnnnnnnmnnnnnooopppppqpppppoooooooonnnnnnoonnnmmmmmmmmmmmmmmmmmmmmllmmmlllllkkkkkllkkllllllllllllllllllklklllllllllllllmmmnnnnnnnnnnnnnnnnnnnnooooooopoopoooonnnoooooopppppppppppppppppppppppoooooooooonooonnmmmmmmlllllllllmlllmlmmmmmmmmmmmmllllkkkkkkklllllllllllllmlmmlllllmllllmmmmmnnnnnooooooooooooooooooppppppppqqqqqpqqppppppqqqqqqrqrqrqqqqqpppppopoooonnnnnnnnnnnnmnmmmmmmmmmmmmmmnmmmmnmmmmmmmmmlllllllkkkkkkklkllllllmmmmllllmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmnnnnnmmmmnnnnnnnnnnnnnnnnnmmmmmmmnmmnnnooooooooppppppppppoopppoopooonooooooooononooooooooooooonnnnnnmmmmmmmmmmmmmmmmmmnmmmmmmmmmmmmmnnnmmmnnnnnnnooooooooooooppppppppppppppppppppoooooooonnnnoooooooooooooonnononooooonoonoooonnnnnnnnnnnnnmmmmmmnnmmnmnnnmnmmmnnmmmmmmmmmmmmmmnnnmmmmmmmmmllllkkkkkkkkkkjkjjkjjjjjjjijiiihhhhhhhhhhhhhhhhhiiiiiiijjjjjkjjjjkjjjjjiiiiiiiiijjjijjjkkklmlmmmmmmnnnnnoooooppppqqqqqrrrrrrrrsssssssssssssstssssrsssssssssssssssssssrrrrrqqpqppopooonnnnmmmmlllllllmlmmmmlllllllkkkkkkkkjkkkkkkkklkllllmmmlmmnmmmmmmmmmmmmmmllllllkkklkklllllmmmllmlmmmmmmnmnnmnnnnnoooooooooooooooooooooooooooooooonnooonnnoooooooooooooopppppppqqqqqqqqrrrqrrrrqqqqqqqqpppooooooooooooonnooooooooooonnnnnnmmmmlmmmlllmmmmmmmmllllmlllmmmlmllmmmnnmmmnmmmmmmmmmmmmmmmmmmmmnnnnnnnnnnnnnnnnnonnnnooonnoooooooooopoooonnnnmmlllllllkkkkkkjkkkkkkkjkkkjjjjjjijjjjiijjjiijjiiiiiiiihhhhhhhhhhhiiiijjkkkkkkkkklmmllmnnnnnnooppoppppppoooopponooonnnonnnnnnnnnnnnonnopoooooooonnmmnnmlllllmmklmmllllmmllmllllllllmmmnnnmnonmnoonoppooopqpqqpppqqqqqqpqqqppqqqqrrrsssssttssstsrrrsrrqqqqqppppoooooonnnnnnonnnnnnoonnnoonnnnmlmnmllmlllllklllllllklllkkllllllllmmmmmmmmmnnmllkllkkllkklllklkklkllkllmlmnmmnnnnnnnnmnnnmmnmmmmmllllmmmlllnmllmnnnnonnnnonmmnnnmmmmmmnmmnnmnooonnoooopqppopqpppopoooooonnnonmnnnmmnnnmnnmnnnnnnonnoooooonnooonnnnnmnnmmmmmmnnlmmlmmnnllmmmmnonmnooooonnnonmnnmlmnmllnmmlmmllmmmmmmmmmnmmmmmnnnmmmmmllllkkkkkkklkkklllkklmmmlmmmllnnllnommlmmllmmmlmmmmlmnmnmmnnnnnonnnnooonmnnnmmnmmlmnmllmmlmnmllnnmmmnnmmmmllmmmllmmmmmmnmmnmmnonmmoonmnponoqpnoqpoopqopqqoopqppqqppqqqqqppqrqpprqopqrqoqqppqqpqqpoopppoooononnmllnmmmmmmnmmnnnmmnnnnonnnnnmmmnnnmmmnnnmnonnnoonmmoonnoonmonnoooomnnnnmmnmmmnommmnnnmmmnnmnnmmmnnmmmmmmmmmnmmnnnmmmnnmmmmmmmlmmkklllkllkkkllllllllmllllllllmmlmnnmmnnmnnnmnnnnnmmmmmommmnnmlmnlllmmlkllmllmllmmlllmlmmnmllmmmlmmlmmllllmmmlmnmmmnmmmnmlmnmmlmmmllnlllnmkkmnllmmmmmnnmlmnnmnnnmnonnooonoppoooppppppppqpppqpooppnooonopnnoommoommoonmnonnonmnoonnoommnonmmonmmnnmmnnnnonmnonmmnommnonmnoomnonnmnoommmnmmmnnlmnnmmmnmmmmnmmmmnnnnnnnnnnonmnnnmmnnmmmmmlmnllmnmllnnlklnmlkmmlklmlklllllkklmmmllnnmmmnmmmnnmmnonnnonmnnnnnnmmnonmoponoponooonnnnooonnnoooooooooooonnoppooopoopqpooqqppqqopqpoopppoooooppnnnooonnmmnnnmmmmmmmmlmnmmmmmmllmnnmmmnnnmnonnmoonmmnonnnnnmnonnnoomnnnnmmmnmmmnnmmnoonmnnnnnmmmmnnmmmmmmmmmmmmmnnmmmmnmmmmmmmmmmllllmmmmlmmllmmmmmmmmmmmmmmmmmmmmmmmmmmmnmmmmmnmmmmmmmnmmmmmmmmmmmllmmlllmmlllkklllkkkkklllkkkklllllmllllllklmmllmnmnnonoooooooopoppoooopoonopoonopooooooopppppppppppppppoooonnooonmmmmnnmmmmmmmllmmlllllllllllmmlmmmmmmmlmmmmmmmnmmnnnnnnnnooonoooppppppqqpqqqqqppppppppooooonmnnnmnnmnnmnmnnmlmmmmmnmlmmnmmmmmlmnmmmmmmnnnnmnnnnnnnnnnonnonnnoooopppppppppppqppopqponnnnmmlmlkkkllkkllllllmlmmmmmmlllllllkkkkjkkkkkkllllllmmmmmmmlmmmmmmnnlmmmllmmmlmmmlllmmmmlmnmmmnnnnopnmnoonnoponnopomnnonnopomnopomnopnnoppnorrqnprqqoppoonoopnlmopmjlnpmjnqpljorpkkprnklqojinrnhinpjhknmiilonjimpollprpmlnoomnonoljhnrmfgpsnhjrsnjlrsojkrtnhirskcisqkdmqkfhpojlmrklsvshowwmeowujentrienpoghosrjhnrqkkqtskputnnusmipwmcdorhcirqfejsujmoxwlmlroinllkileeeemlkhgtqnhkysnhp~wrkoxpmglwpmhimbcbdhchnkkfpzqno\x80\x89zpt||mgpspeaegdaaed]Z`hmiisxx~\x81\x96\x98\x91\x82tyzn^\\ovl[^hgYDN\\b_Wdpnmu\x90\xa1\x90\x82\x90\x9d\x93qav\x81wad|yhZYg_OKRglbdehbY[fx\x8e\x94\x92\x97\xa5\x9a\x86nole_emnglke[PRNNVXab]ZU[fy\x94\x99\x98\x92\xa3\xa9\x88o^km_Ufwygdhk\\GHP[WS_ieUQam\x7f\x8d\x97\xa3\xa6\xb7\xaa\x8c|ujaP\\jlkfppeTIKNNMU`gd]_ju\x86\x96\x9e\xa7\xaa\xb7\xae\x90\x84ynbSZacefmpl^TNJJCJS`b^gsv|\x87\x9d\xa5\xa4\xaa\xac\x9a\x8fymdRUVV^]kplbUSI><AQZW`mvu~\x9b\xa4\xa2\xa0\xaf\xb6\x9d\x86yrhODPWUSctpb^aXG<GQMJYrzu\x81\xa2\xae\xa5\xa3\xb6\xbd\x9e\x83zvbDCNVPUmvxmiiYL@DGBNZeou\x86\x98\xa4\xab\xac\xb6\xb4\xa4\x95\x7fpZE?:<@Kafkrtub[ZTMHR\\X_n\x84\x9a\xa0\xaf\xbd\xc3\xc5\xaf\xa0\x90s]C7/)08NZhx{\x80vnj\\UKNRNZf{\x91\x9b\xaa\xb6\xc2\xc3\xb1\xa4\x92~cH<0-)0EP_kw\x82xvpjaRUWUYav\x85\x90\x9e\xa8\xb2\xb3\xa8\x9f\x93\x85pZPE>47EKU]ivuvwxsgdda]`p}\x82\x88\x90\x9f\xa2\x9a\x96\x8f\x89yh\\TMA<ELV]gux{yvqkb^\\^^amsvy\x82\x8c\x89\x84\x88\x89\x84xonj^Y^giehqrpikkg__^b]\\finkpz}|}\x85\x89\x80\x7f\x7f|ulnqmmklmfd``\\WWVYYX[aegiqvvw\x7f\x83\x84\x81\x84\x86\x82||zwlkifecdccaa]_ZWVX^^blmuv\x7f\x86\x87\x86\x8a\x86\x89}{{uqlkoilhdb^[VSVQTV^_enuxx\x81\x86\x83\x80\x83\x82\x80yx\x7fy{xyzrpjd_XXXQTRYX\\cmmty\x80\x82\x82\x85\x85\x83\x80~}zzyxwrrnhb[WTOQPQVY`flsv{~~\x7f{~|y|zz{{~ywsnia]TQOMMOS\\afotz||~\x7f}{}}~~\x7f\x83\x7f\x7fzwrhc`WSPRTRXbcinuxu{|zzz{{z~~~\x80|~xurij^\\XVTTWa^cmnsry{vx{xyy|{z{\x7fxztvnhhaY[UZRY``bkmsqu{vy{{}{{\x7f{x}zvsnrf`aZXWP^TZ_^mfkvuyw{\x81{}\x81\x7f}z}}vsssjdfa^WXZVXX]dbilsxt\x81\x84~\x80\x82\x89\x80r\x83}pmjqcZi`]ZZbXX``balnnvw\x7f{\x83\x88\x81\x88\x87\x85\x83y}yiidaZQXWQQX^Y^fijjvuz\x84\x85\x8d\x91\x92\x98\x90\x90\x8c{yodYQPNGISWR\\^jecroop{\x88\x81\x89\x9a\x9a\x96\x95\x97\x91|sj^JAB>=BJXZckqqrmry}|\x87\x94\xa0\x9b\x9a\xa5\x97\x84sf[=4649;Occosv{lkhpyt\x83\x9f\xa9\xa9\xa9\xb5\xae\x8fxmY:&-/+2I]aetvskdmut{\x8e\xad\xb4\xaa\xb7\xc0\xa4\x85rfC!!\'  :U]`q\x81xokt~y\x80\x96\xb2\xbc\xb6\xbf\xc6\xac\x87q_?\x1a\x14\x1d \x1e2Ybdp}|g^iont\x92\xb1\xbd\xbf\xcc\xd0\xb8\x97\x80gA\x1f\x1b\x1f\x18\x1d7W^]pzo[Uhibt\x9e\xba\xc0\xc8\xdf\xda\xb0\x96}Y+\x0c\x11\x0e\x07\x169UWcz{m]jzms\x98\xb9\xcb\xc6\xda\xe1\xbd\x9b|\\8\x0c\x0b\x0f\t\x172Ub\\v|hXShb[\x86\xad\xc1\xcc\xdc\xf2\xd8\xaf\x9byN \x14\x1a\r\x0c)LZSf{lPSnl^\x81\xb3\xd1\xc9\xd6\xf6\xe1\xac\x8esL\x16\x04\x0f\t\x04\x1bEXSaxrSTrlf\x85\xb8\xd5\xc3\xdc\xf4\xd2\xa9\x84lJ\x14\x0e\x17\x13\x16*M`Y_h_F@Z^g\x8b\xbd\xe0\xe3\xf4\xfc\xe0\xbc\x8cdB\x1a\x08\r\x10\x1d3Mdh_cXCJJTj\x89\xbe\xd3\xdb\xf7\xed\xd8\xb4\x80c;\x19\x12\x10\x17$?[e_``I>LGXq\x98\xce\xd7\xed\xff\xee\xde\xaav\\4\x1a\x0b\x0b\x1e)AVa^]TA@JIb\x84\xb2\xd5\xde\xf8\xfc\xe2\xc2\x92lK"\x17\x12\x1a\'1OaXVMC47?Og\x98\xc2\xdc\xef\xfe\xf9\xdc\xb3\x81a:\x1c\x11\x0e *;SW]TD>FNLf\x8d\xbb\xd9\xdf\xf7\xfd\xe2\xc1\x8enQ#\x18\x0f\x1a*.DYYWKACHGRv\xa3\xc9\xd5\xe6\xfa\xef\xcd\xa2za:\x1c\x16\x18),6MZWLCEUQTv\x9e\xca\xd7\xdc\xf7\xf4\xd5\xac~iI\x1f\x11\x0f\x1e# 7LOJCI^`Xx\x9d\xc3\xd3\xc9\xea\xf3\xd0\xad\x86t_1\x1e%.,$4TJ;CEOTQt\x8f\xa9\xca\xc8\xde\xe9\xcc\xba\x93vf<,*(,)8LJAHNNY_o\x91\x9e\xb4\xcb\xc8\xd4\xc7\xa9\xa2z`V87:0:BHUGBRJGT[q\x89\x8e\xb0\xbc\xc2\xcf\xb2\xad\xa4xnZFL@ARKS[JJHEFOVa{\x8c\x9f\xae\xb9\xbf\xb3\xaa\x9c\x81pdTHEHGEKJBL>>HIYcv\x94\x9d\xad\xbd\xba\xb7\xaa\x96\x8dzbVUMOGHXMGHAEDAP[j\x7f\x94\xa5\xb3\xbd\xbe\xba\xad\x9c\x89xiZQROORNWOLLKNNQcs\x83\x91\xa0\xaf\xb2\xaf\xa6\x9d\x8f{lf]WTW[VURRJCEAJRa{\x82\x99\xab\xac\xb5\xac\xa2\x9a\x88yp`_\\Y[\\Z[\\PKJEEESgm\x7f\x93\x9d\xa9\xa8\xa1\xa6\x97\x88\x7funh_`^`\\VRMJI>DNOenu\x8e\x97\x97\xa0\x9a\x94\x90\x85|tjgh]ZbVUTDLEBJIZch~\x87\x8e\x97\x95\x9b\x97\x8c\x8a\x87\x7fwsooha\\ZRKGGLKMWbmw\x80\x89\x93\x95\x96\x95\x96\x92\x8b\x88\x82{yslic`ZUQUQRVVafgpx{\x82\x80\x83\x86\x82\x82\x80{zzuqlldc\\WZVYXZafhntvy{}|{~~zxywupmkid_`_^^^dehlmrutvurtqpmlnlekngjknjjpljrnqunsuppkjmabda_addgfonostz{u}\x7f{zx{tmojff^ae]^acbaelnpv\x7f\x7f\x81\x84\x84\x87\x80~{wrmhkd]b^Z]YY`Z`ihovy\x84\x81\x82\x8b\x86\x85\x85|\x83ztsmng\\a]UVNVVQW_`imv~\x7f\x84\x89\x88\x88\x88\x85\x81\x7f{wtnihd^\\[YWRWYV[^_fhnsv}\x80\x81\x88\x84\x83\x87~|{sokhf__`YZZXYXZ^`^gmmtv}\x85\x7f\x84\x8c\x88\x88\x85\x85\x83vwrge^\\\\TY[[Z`dccgijfkpoz{\x83\x90\x8d\x96\x9b\x93\x97\x8c\x83\x7fmg`SUUOWX^a\\fe]c^Zachox\x89\x91\x94\xa7\xa3\xa1\xa0\x91\x8bvfaMHIBKNRbahklqfej[Za_`i{\x84\x8a\x9c\xa9\xa6\xa4\xa1\x99\x7frfPE>?BCS]fos{vlq`WXOQS]qt\x8d\xa3\x9f\xb2\xb9\xa7\x9f\x8f\x7fbPK82@<H[]ktrunkbUZHN\\Wo\x83\x92\xab\xb2\xc5\xc7\xb6\xb5\x9b\x80pNE5+33?PWgqtuvqdc^OOZWbx\x87\x9a\xae\xbe\xc6\xc4\xc0\xb1\x97\x82gK@,&/-:PUesuyxsi`]NQ\\Sj}\x87\xa5\xaf\xbe\xc7\xba\xb9\xa1\x86vRA5 \'+0>Q[fxw|\x81usm`]P[UXus\x8f\xa6\xa7\xc2\xbb\xba\xb1\x96\x8ahSJ*//*<GP`ktyz}tkn]VbW[su\x86\xa0\xa5\xb5\xba\xb8\xae\x9c\x8eo]O45.):<JW`mqzvxyfneRdXXsn\x85\x95\xa1\xaf\xab\xb7\xa4\x96\x92jeW8C75DCRZeqr~|z}oof[fV^qg\x81\x92\x90\xa8\xa9\xa8\xa4\x9a\x8fvo`FOA<JHO\\chqxvz}osl\\aWU^\\itz\x8a\x8b\x93\x94\x8a\x8b{rn\\\\VOVRWa\\jolxwswqjfd^[`^bpn{\x84\x83\x8d\x8d\x88\x89\x83|vskfibdiflpnqtonniefa^b`bhfnqowwtxvrupoolkmnmlnjjkdeb^_]`bcijnqsvvvxuuusrqrpopnqpqqppolljhiggihjmnqruvvwussrqpnmmlmlkmkkmljljjiiijjmnnpqpqqonnllkjljjkklklllnnoqrqrqonmllllnoqprsrsrponlkkjkjkkllmooopppqooolmkiihhiijklnnoooonnmlmlnnnqppponmkkjkkkkllnnopppqppooonnnmnnoqrrsssrqonmllkjiijjllmnnprqrqqqpooonnnmnnnoooonnnmmmlmmmmnmnnnnnnnmllkjjjjjjklllmnmnmllllmmmnooooooponmlllkkkkkkllmnnoonnnnnnnmnmnnnonnoonnnnnmnmmmmnnnoopooonoonoonnnnnnmnnnnnnnnnnnnnonnnonnnnmmmmnnnoooopoponnnnnnnmnmnnnonnnnnmmlmmmmmmnmmnmmmlllllllllllmmmlmmmmnmmmmmmmmnnnonooooonoonnnnnnnnoooooooopooooonnnoonoooonnooonnnnnnnmmmmnmmnnnonnnnnnnnnnnnnnmmmmmmmmmmmmmmmmmmmmmmnmmmnmnmnmnmnnpowd\x8dik\x82F\x84S^gOscorjxqmydoqfyb\x83xs\x81`\x7f`kk\\ianduhunluerhlpdvir{g~\x81p\x87tljfYcZbjbyo\x83s{|kzcnfeldphsrpwpwrtsqopmonhtcoojopmqpnqqnppkrlkpeoihmikklkpkgrhftanncrjlqpoqumtrjukoojpkimdnehm_t[oj^y`i}b~|r\x87~y\x81vnkj\\]gOdfVofgjtarw]\x87ox\x8e|\x91\x89\x8c\x8ax\x84ccbBXHG[Tekq}u\x85yu}dijRf]Zvj}\x92\x89\x9d\x9a\x97\x93\x82ycSL8?2BOLspz\x9b\x83\x93\x8b\x82tahKC]QS\x84\x81\x8b\xc4\xa4\xb6\xc0\x8f\x8emS5#-\x10/AHx~\x94\xa5\xa2\x9e\x90\x8beaW8?B<Dfh~\xa9\x94\xcb\xb9\xa8\xc4~\x87d8H\x14)0&YXu\x8a\x8d\x9b\x8a\x8ctd\\DB9@?O_r\x89\x90\xb4\xb1\xbb\xbf\xa1\xa8\x7fkV84($;FYg\x88\x8c\x8d\x94\x8bwg_H8HC>Unjr\xa5\x96\x9d\xc9\xaa\xb6\xb2\x9a\x92ngM<B8ERjjz\x95\x83\x89wxgEUIDLRi\\l\x87n\x90\x9c\x8a\xb2\xa4\xab\xab\x8d\x9er[a;9><DUdfp~|eghJIPPPPnb]\x82w\x84\xa2\x97\xb2\xb8\xbc\xb6\x98\x9ay[Q6<76RWbuqqtbYKLH@RYdihz\x82{\x82\xa0\xa5\xa5\xb6\xbd\xc2\x97\x91\x90\\MDA;4QX]cepfYWRJGL]bau{mr\x87z\x80\x95\xb1\xb5\xa1\xc0\xc2\x8a\x80|^@4K@0Uj]YqrVL\\P:Iaa\\i\x80mb\x82{\x84\x9b\xa9\xbb\xaa\xbf\xc4\x8f\x86|dH7NF<L^gUVi\\@FQI>Rif_uwlz\x82\x92\xa4\xaa\xc0\xb9\xb5\xc3\x9f\x86zh]=IUIGUiYJe_KGSYGPoe^pupp\x81\x89\x8d\x9d\xb5\xbc\x9d\xb8\xb9\x87yrtM?\\SJKYbLSYKFCJJK[bifdvty\x7f\x92\xb0\xac\xa6\xb9\xba\xac\x88\x7f\x81cWMT]LLYbVFRXH=IVUOdmeir|\x7fz\x8e\xa5\xac\xa7\xab\xba\xb4\x8e\x81\x83v]O\\cMK[ZXJSTGLOLZ\\ge_srky\x84\x90\x9a\xa7\xb4\xb1\xaa\xa9\x97\x85rfeSOZWPXXVSPSHGROUYeibnrpz\x85\x9a\xa6\xa9\xb4\xaf\xb3\xad\x8c\x83ymbR[]XSZ^RTUNLGUQO_fh^j}ju\x83\x92\x9c\x99\xb2\xb2\xa6\xa5\x93\x8dwfi\\XYYYXVVTMMIHNHXadb^uniu}\x8f\x95\xa1\xb0\xa9\xac\xa3\x9a\x87upc_YW_X[ZUXRSIHROT]ig_nnqv\x83\x9d\xa3\xab\xab\xac\xb7\xa1\x8e\x81ztX^cb[RfaNNTZFASWTRcmcajwy\x80\x98\xa9\xa9\xa5\xac\xb4\xa1\x89\x82|p\\[`\\YR^\\PRVWKJUTSWbd`elvw~\x92\x9e\xa1\xa0\xaa\xaf\x9a\x8a\x86\x7fjZ[_VMVaZPU_UGLTTJQ`b]]owu\x85\x99\xa3\x9b\x9f\xad\xa5\x8f\x81\x80v`VY[UNU]VPSXSKPXZX[ehffny\x80\x8c\x94\x9b\xa0\xa2\xa2\x97\x8e\x87{oc^^ZVW`a]]bb][_ba`djjfgnppw\x80\x85\x88\x8f\x94\x93\x90\x8f\x8a\x83{upmeab^[YYXUVXXYY]babeims{\x82\x88\x8f\x96\x9a\x98\x97\x95\x8e\x85{tne_]\\XVWYWVWYZYZ\\^__cginuz\x81\x87\x8e\x90\x90\x90\x8c\x85|smf_\\\\[ZZ[[ZYYZZZ[]_adhls{\x82\x88\x8f\x94\x96\x94\x92\x8d\x85{rke_\\\\\\[[]][\\]_bdfgijjklmptvz~\x81\x82\x82\x84\x82~|xuqmligfedcbbcdefhklmorsx~\x82\x85\x88\x89\x89\x85\x80ztmf`^[[\\]^_``___`adfhjlmoqtx~\x83\x86\x89\x8a\x89\x85\x7fyrmgcaabcefghfffeffiklpqrrrrqqqpponnlklkjjkmmmnpppqqrsrponmkkjjiikkmooqrrstutsutssrplhfca_`bdgjnruy{~\x86\x89\x88\x89\x87\x82xoh`ZTTY\\`dhkkmljgggfhkmoqstvvuuuusrqpnllmlkklmmmoqrooqponnnlkjjjiijjjjlmnnnopppqqqononmlmmmnooppopqrrqqqpopoonnnonnmmkkjihhiiijkkkkklmlmoppqqqqonmmkkkllmmmmmnmnmmnmmmnnnnonnnnnoonoonnnnmmmnmmnnmmnmmmmnnlmmmnnooopopponmonmmnnmlnoonoqonnnnmmmmnooooopponnnjknolglojdhqvoq\x85\x8b{hkwlYWhui`hwt`]nvlco{wkjstkeiolhinolkmomjlonkmqrnnqrolmppkhknlijmuunfqxochttigotolegnleTl\x8f\x89rl\x8d\x97lTcymO]xwf_ougdinporstvqklqrhfotohjpnkhkolknstrssrutswyuoptlffkjc`ghgdfmkhlpqhjpokgkrmjg}\x8cnbj}qU^mm`Whohak\x87\x91\x83\x83\x8e\x8fziij\\X_lieotrkjroaagnlglg_X[b_a\x80\xa0\xa3\x95\x89\x9f\x98iTRc\\K\\y\x83uiy{hXZgb^kv~uprmggc`\\_c^XUr\x87\x88\x86\x8c\xb3\xa7\x81vv~cP_plcn~tfcnpb]fkfafic__WT[_`]q\x8e\x92\x97\x93\x9f\xa5~lnl_O_nhciuo[Wbe\\Wbkicdlh^\\XUZ[bi\x80\xa2\x9f\x96\x9b\xa6\xa3ydki]NZplgjup][ec\\\\gnhmsrkccZPV_cft\x97\xa7\xa0\xa1\xab\xae\x8dpnk`OWpnlt\x80}ecj`RTdf[bv{ofjkVHP\\ck\x8a\xab\xac\xad\xb1\xb6\xa6yi`SF=S`bmz\x83ujkdVNQWUYeprnmk`UST]du\x97\xa6\xae\xb6\xb8\xb7\x97xjVB14CKXh{\x83}{ui]QMJIR[djlqogc_afq\x89\x93\x9a\xa9\xb3\xb8\xa7\x91\x84s^F8::?JXku~\x82~{tjc[[]^bfkolmkihflsv~\x89\x90\x93\x90\x8e\x8a~rf_XTTY^fmpppqpjgjmnptxyurolga_aabeiprswwwtqromnmnppomnpoklmnmkklkihhjhfecbacdgloruvwwwvusqqpomkkkkjklnonmopoopqrqpnmmllnnopqrstuutsronljggiijkmooomlljjloqrrsrqnljhfefgilnprrqqomiggghiknpqssssrrrrsrqpnmkiigffefgjlnpsuvvutqmmllkknppqqqqonnnmnnnnmlkihfeeefilnortuttsrpnmmmmnnnnmlkkkkllmnoopqqqooonnnooooppqpqmyxllkokmrsvxpmb`[YVZhlpux{xzzrpnmidfbafhlmmqrqrrrprspqpqonoonoqpprtsuuuzyzxxueif^`acfhopourqtrpmomjmlklmomlzjmuklkjgbggaeggfgkfgkgfimmpxyy\x80}yzuqkgf`adafhklhqkimggbfibkppz\x7f\x85\x8a\x8a\x92\x8b\x86\x86zoh^WLMPKU__jqtxx{yorneecfefuuz\x8b\x8b\x8e\x93\x93\x8d\x81~t]]TEJHHOVacp{v\x82\x87{\x82}poh^W_ZVmjs\x8c\x8b\x97\x9f\x9f\x9b\x8f\x8dxgcNDF@BHPWakv{\x7f\x87\x83\x80\x7fwkdaTMXNUki{\x90\x91\x9f\xa7\xa3\x9b\x97\x87nkWAH<8HGP_ioy\x84}\x82\x88tvw^^aJM[I\\oi\x83\x93\x93\xa4\xa7\xa0\x9b\x95\x7fmkMFM3EOA_dayxy\x80\x80ytue]aNQ_Mfup\x96\x97\x9e\xb3\xa7\xaa\x9f\x90\x81laJFB7GFN_dnx|\x7f\x80\x82{wsja^YMXUUnl~\x92\x93\xa2\xa8\xa3\x9f\x98\x88xlZNHCAFNP^dgvst\x7fpwpgh[[WY[aqp\x8c\x8f\x96\xac\x9c\xa5\x9b\x8c\x80nbNMA<K?QXXljsxt|tpsf`hRZcQmpn\x90\x8a\x95\xa1\x9c\x9d\x8f\x91|hpNPR@PQPdaitrwyxvqngc^Y`]`sp\x81\x91\x8e\xa2\x9c\xa1\x9c\x8e\x8dtmeMUKHSP[`gqn|ws{qgp_]_W_[ioo\x8e\x82\x8f\xa4\x86\x9e\x92{\x85ldaMVJMVR_deqqsxsrqhhd\\abairt\x80\x88\x8a\x8e\x94\x8c\x87\x89uprX]^M\\ZWcbgimniqjgldefbdgfinppuxtxyssunnoijkhfkkhpnmtqovooqllkmlksmqvpsuourprqpnqnnomnolomlmikjinlmpnqqppsoornmqmoplqpnqpnpomnnlnlmnkmmkmmmnnoopooolnljnkjmlkmkllmlmmmlmlllllllllmlllklkjkjklklllmmmmmnlmlllkllkmllmlmlmnnooonoooopoppppppooonoonooooooooononopoppnonnnnnnooooooononnnnmmnnoooopoooononnnnmmlmlmmlmnmnnnnmnmlmlmllmllllmllmmnmmmmmmllllllmmmmmmmmmmmmmlmmmmmmnmmmmnnnnnnnonoooooooooonnnnnmnnnnnnnnnnnooooooooooooonnonononnnmmmmmnnnnmnnnnnmnnnnnnnnmnnnnnnnnnnnnmmlllmmmmmnnmnmnmnmmmnnnnnnnnnmnmmmmlllmlllmmmmmmnnnnnnnnnnoooooooooonnnmnmmmnnnnnnnnooooooooonooooonooonnnnmmmmllmmmnnnmmnnmlnrpim\x90\x85QV\x94\x8d=8\x8b\x8dCH\x93\x92IV\x9c\x92KD\x94\xa1R2m\x97l;[\x87{hk\x85\x90oU^ynBBq\x7f`[\x82\x96xgw\x82mV`ql]c}\x7fso{wjmkihljivxegyuabusdiwphlmnnpnny\x89z_h|qTYxs\\k\x85vgy\x84hXpzh^o|l`p|qbetvsnsof^`ki^_pvjhx~mcowj]jvpegy\x85sdmyya`prmgmwstudl}xh[}\x88ZOy\x8cmMp\x88iTh\x81sQc\x8auVY\x7f\x85ZYz\x84l^y\x89ibuvjYk\x85k^eus\\eureejfemklfu~qz~\x87\x84s}}nmefZMX\\a]Zedbedgjhox\x89\x8e\x91\x90\x98\x94~wrc[\\da_cod`sbXY]jSXrjefnzt}\x8b\x85\x94\x97\x86\x81}|qd_c[hfej]fcMVakd[nwi`by~z\x87\x8b\xa4\x99\x85xzzaWbtiRk~l\\OjlLTgoj^g}cagz\x8e\x85\x85\x8e\x96\xaa\x84i\x82}kGUunV\\|}YTbhXMg\x80cWfwl\\i||\x84\x8c\x93\x92\x91\x9e\x8cehwiV\\qm]isjW[a^HZri^`spRepmlx\x99\x90\x83\xa9\x92~lm\x84]QntmY`|aOQW^OZodgcVd\\]in\x88\x8f\x8b\x8e\x90\xa3\x90ovt{`Stzh]o|YMW_UE^waVWkkQg|\x88\x8f\x88\x96\xa4\x97\xa0\x86x\x81jbciqfevnWSZ]KL_fZTfjY_noos\x89\x8e\x90\xa1\x9d\x97\x90}ollc^fve[hbXQNSRX]\\`h]]cfqmz\x93\x89\x95\x9b\x9b\xa3\x84uoslOa\x81kXd~\\<bcEMcmRXv_Whifly\x8b\x8c\x8d\xaa\xa6\x8b\x92\x8d\x83g_oicdcp_U^XPHO[P\\g_]accfy\x86\x8e\x98\x9a\x9c\x9f\x98\x8c\x82\x80pbecaecjh^``VPSXWae^^ab_fw\x84\x8e\xa1\xa3\xa0\xa5\xa3\x97\x81|sj`ahbbf_\\T^UEV[T\\`gc[fccpq{\x8e\x8c\x9d\x97\x95\x9b\x89ysif\\]gZcdW`cTSTYNQjf\\hof`w~\x82\x9c\xa4\xa6\xa2\xa1\xab\x81uweYMdfR^kcT\\cQO[[\\djhbmhbpvz\x8d\x98\xa1\x99\x9e\xac\x87yyk^Q`dSgj_\\dcWRZSP`ddehoen{\x82\x92\x9d\xa3\xac\xa2\xa4\x95\x81{\\]_LS[\\b\\al^X`YXXbb_jf`nrs}\x87\x98\x9f\x99\xa0\xaa\x8d\x85\x7fhbXT\\Y[agb_i_TXROR[__cpnm|\x84\x8c\x97\x99\xa4\xa0\x9b\x9b\x80~lU_NITT]\\`me_d_ST^ZQdn[i}sv\x85\x92\x96\x95\x9f\xa7\x95\x8a\x8cm_eKFTVT\\pqhslcYSXOMZ_\\iqt\x7f\x8a\x99\x9b\x9d\xab\xa8\x9d\x92\x8ay`[P@GIIXafotvpjf^WTTUS`fkz\x84\x90\x98\x9c\xa7\xa4\xa2\x9d\x92\x84vkYPJDGKPVfmlvrqjef\\[a[delru\x85\x84\x87\x90\x8f\x93\x8c\x89\x89|vpa]VQQRS\\aclnnonmihfchgelprqu}y}\x80\x81\x81{\x81xlsicc_e__habfdfbdgbdjbfleonq~n\x84\x82|\x89}\x87\x7fv\x7flnm^i_\\f]acaheiofkokjnlpsrwvx\x83|\x80\x85\x80\x83ytvhgf`cbbfcggfhhhgejghlixtq\x80x\x7f\x82{\x86{y~nqnhiciedleflffefi`hm_kpenmsnj~sp~|yv|xmsrfijgeffbbcc\\cg_fghpfw}r\x81\x88\x83\x86\x89\x85zvsb[\\QQVY\\enqsxwrljk^_mbhuu|\x80\x86\x89\x82\x89\x82v{oeg^]][abhmnvrswjgh[^Zeae\x7fu\x82\x98\x8d\x94\x96\x91\x87xwfUZOITU[entx}~wsl`XSNSY_r\x82\x8c\xa3\xa6\xa6\xa9\x99\x89w`P>866ER]r|\x82\x8b\x88\x82|rd\\TLJHSTbyz\x97\xa0\x9e\xb2\xa1\x9a\x92wnWLI;HNQfov\x83\x86\x85\x83\x80tkhYTRJQO^ik\x8b\x8d\x91\xa7\x9a\x96\x91\x7fp]WH@IGQ`hs\x82\x83\x87\x8a\x81{wd`XNLPZZt\x84\x87\xac\xaa\xac\xb8\xa2\x95\x82kSA<+1<AWft\x83\x8a\x8f\x8d\x89\x80vm]YQGMJI^_k\x86\x87\x9b\xaa\xa7\xae\xa9\x9b\x8c{gQK=8ABO^jv\x82\x88\x87\x89\x81vrd[WNLLLOU[dlv\x80\x85\x93\x93\x95\x98\x8c\x89}nhWRNHNOX`ftr~\x81z\x85ztxee`URWZWqt|\x9c\x96\xa1\xac\x9c\x97\x89v_RH5=AAV`k{\x83\x88\x8c\x8e\x87\x83\x80snj]ZXQPSSW`dnz\x7f\x8c\x91\x96\x98\x91\x91\x83ysa]VPTT\\agrrz{wyqkh^ZWTSVa_q\x81\x80\x9a\xa1\x9e\xaa\x9f\x95\x88vdOH<4@AJ]gs\x80\x88\x8b\x8d\x8e\x86\x81}qid[TSOMPSVakt\x83\x8e\x98\xa1\xa4\xa4\x9d\x94\x86reTEA;=DLYdr}\x84\x8c\x8c\x8c\x8b\x81yqeZSJEDEPVfw\x82\x9b\xa3\xab\xb6\xad\xa7\x9a\x84q\\K<557BN[jv\x81\x88\x8d\x8e\x8b\x88\x80vncYQJDEDIS\\l|\x8a\x9b\xa6\xad\xb0\xaa\xa1\x91}lVG<46:BQZjw}\x88\x89\x89\x89\x82|sj`WQIGGDPT`s}\x91\x9e\xa7\xb1\xad\xa9\x9d\x8bzdVG<><COXcpy\x7f\x86\x89\x88\x89\x86\x80zsjaZQKJFLT[l{\x8a\x9b\xa5\xae\xae\xaa\xa0\x8f\x7fkYNA<?@IS\\gpw~\x83\x86\x86\x87\x84}ypd]SLHFLR^n{\x8e\x9d\xa6\xaf\xaf\xaa\xa1\x91\x82o_UGDDCJPW`gpu{\x81\x82\x84\x83}ypg_VQMJOT\\it\x81\x8d\x95\x9b\x9d\x9b\x95\x8b\x80sh^UPNLOSW]cinsx{}}zxrlf^YUSWY_iq{\x85\x8d\x93\x94\x95\x91\x8a\x83ypia\\XVVVX[^chnsw{|{yupjd^ZVVVY^dksz\x80\x84\x87\x88\x86\x82~xsniec`____acfhknpqssrpnkifeeefijnruy|~\x7f\x7f~}{yvspmjhfeeeghjlmmmmllkkkkllmmnpprtuvwxxxwvtrqomlkiiihijkmmnnoooonnnmmmmnmnnnnnooonnmllkjjjihgggggghijjklllllllkkjjjjiijijiijjkklmnnoopoonnmmmmmmnoopqqrrrrrrqpoonnmmmllmmnnopppqpppppooonnnnoppqqqrrrrrqqpopnnmlllkkkkklmmnnoooppppoooonoplnmnmmmmmnnoppoonnllkjjjjkkllmmmnmmnnmmlmlllkkkkkklklmlmmnnmnmnnmonnnnooopopoppopooonnnnnnmomompnoqnqppqqnolnlmmloloooppnpnomnoloknlnknlmnlomomoooonnnnmmlpjomlojnmkokkniohllgohkkjljlnhokllmllmnlnnmnkpkmnlnlmlkmlimkjnhpiolnmomqnnsjulroopplugujppjpojsirmmtguonttpwnvlqpknokqlslsmrpkqnjqjoionllrknqnnosiukppmonnkmphookmonmhsfjogiphoinphqpgsjoojuipojmnhnjjokjrjplonkpnirkomnqhspjtnmqlrioqiopimokkpkkolokrmmsnosppqroptmppmplpoitinlloimpdpifqforjyquzqyspvimockhffdiadibgjlmvxy~\x81~|}xoom`dc]`bebklhqrmssnpnjhha_a][gbhwr}\x86\x82\x88\x87\x82|xnda\\UXXY\\efitsw~z\x7f}|zrskadZVZY\\cnt\x7f\x8b\x90\x93\x9a\x96\x8d\x8d~pm]TUKOQU\\cmry\x83\x81\x88\x8a\x84\x85\x7fwoh`VUNLPIZU]rg\x80\x85\x83\x98\x8b\x96\x90\x85\x8csuqWiZTcV_cbljuww\x82\x80}\x85{uvi^^TKUPRffr\x88\x86\x96\x9c\x98\x9b\x91\x87~lgUNPBIOIW\\]jnt{\x80\x86\x84\x87\x87{ztdaZMQIOVTjnt\x92\x8a\x96\xa6\x95\x9c\x98\x83\x7focZMPGESLRd]ivq\x7f\x84\x84\x8b\x86\x88\x82wxhaaQTRKXU\\ol\x7f\x89\x8b\x98\x99\x98\x95\x8c\x87voeUVNIOMP[[elpx{\x80\x83\x7f\x83}utib_USSNZ[avv\x84\x95\x92\x9c\x9f\x96\x94\x87{p^YMCICCRPWggq|}\x84\x88\x88\x88\x84\x83{spe_]VVVU^^frr~\x87\x84\x8c\x8d\x86\x84}tmc_YSUUT[_ahkpuv{}|}zyuqmieca`adhkrx{\x82\x86\x87\x87\x87\x83~xsmiebbaabbcddgiknqsvvvvsrpmlkiiiijjkllmnnooponnlllijiiiiklmopqrssrqpnljhgffghjlnqsuvwxwvvtrqonlljihhgghiikkmmnmnnllkjjjkklmmmoopqqqssssssrqponlkkiijjkklmnooooonnnnnnnoooopqqrqqqpoommlkkjjjjjiiijijjjkklllllllllmmnnooonnnnnmmnnmmnnnooooooooonnnnmmlllllmlmmmnnoooooooonnnnnmmnnnnopoqppqpoononnnnnnoppqrqrrrqqqooonnnnnnnmmnnnnmnooooooonnnmmmmlllllmlmmmmmmmmmmmlllllllllllllllmllmmnnnnnonnnnnnmmmmmlmmnmmnoooooooppnnnmllkkkljkmnnoqru\x7f}oiprkcYWblf\\]lvtoqz\x7f|vx}ztpnkfcbcdcdhnruvwxwwvurpnoomllnnmlmmkkmonnoqqrrroopnljkkjijmnmlloqpllmmmlkkjjiijlkjkllnponopqppommmmnmnmmmmmnmlmnnooonmnonmlkllmnnnoqrsstsqpponmnnopqrqommmllkiijlnomlkkkjhgggimptwy{{{xuqlhdaabbcgjnqstssrqpnkiijjjjhgeeffeehou{}\x7f\x7f~{wqh^VRRV[`hov}\x83\x83\x81|wrnjgghhilpsspmjgecbcfkt}\x86\x8b\x8c\x8c\x89\x83zm`TLHHMWbny\x83\x8a\x8e\x8d\x89\x81xphb^]]`fmrtttqnjgddgoy\x85\x8e\x93\x96\x93\x8d\x83tcQB:9?IWft\x80\x89\x8e\x8e\x88}qf]XWY^dksz|{wph`YUSTW^k{\x8c\x98\xa0\x9f\x9a\x91\x84s`M=418EVgw\x84\x8f\x96\x96\x91\x86ylaYVW[`gov{}zume^ZXY^iy\x8a\x9a\xa3\xa6\xa2\x98\x89v_J7,*2@Tj~\x8f\x9c\xa2\xa1\x98\x8byi\\SOQXakv\x80\x86\x88\x83|rh_XUUYdu\x87\x97\xa0\xa3\xa1\x98\x8awbM<0/8FYn\x80\x8f\x99\x9d\x9a\x8f\x7fn^RKJNV`lv\x7f\x84\x83|si`YTSUZeu\x87\x97\x9f\xa0\x9c\x93\x87wcO?65<IYiy\x86\x92\x96\x93\x8a}odZSPRXaku|\x80\x80}wog`ZWY^jx\x86\x93\x9a\x9d\x9c\x93\x85r\\I:35<IXk}\x8c\x98\x99\x93\x87wi^UPOT^jw\x82\x86\x86\x81zri_WQSXdt\x84\x92\x9b\x9f\x9e\x98\x89u^I:239DUh{\x8c\x95\x97\x92\x86yk^TNOU_lw\x81\x87\x89\x85\x7fwnd\\WWZbp\x80\x8e\x97\x9a\x9c\x96\x8c{eRC:89@N_q\x7f\x87\x8b\x88\x81wk_UPPV_kx\x84\x8b\x8e\x8c\x86~qeZSRS[iy\x89\x94\x9a\x9e\x9c\x93\x84p^OE@?EP^my\x81\x85\x85\x82{rh`[YX[agnt{\x83\x86\x86\x84\x83\x84~vka`acffny\x82\x88\x84\x80}wobXTSVX\\dkqusrmjd^ZY\\enw\x7f\x85\x8b\x91\x8d\x86ylcZVVXajw\x83\x89\x8c\x8b\x86\x7frg_YVTV]dlpsvvtpkf`]ZXZbiq{\x85\x91\x96\x96\x92\x88~o^TJILS_lw\x84\x8b\x8d\x8d\x85}rg_XUWX`glruwysnhb^YX_ekr}\x8c\x92\x93\x91\x8e\x89}nbZUQQU^gnsy\x7f\x80|xurnifhjjknrsqonlgc_^adiow\x82\x86\x8a\x8b\x88\x84|qi_YURTX\\bfkprsrrsqoonppprrsrnnkhebbehkou\x7f\x84\x87\x89\x8a\x88\x82yqjb]XXXZ^beilnpprsrrpqqonlkieeca_]^bdiou~\x84\x87\x8a\x89\x87\x81xphb\\XVWZ]adjnqrtvvutqpponnnnlkigfedfilosx\x7f\x83\x85\x88\x87\x86\x82zvpiea_^^acehknooqqppmnnllllljiihffdddfilosy}\x80\x81\x82\x82~{wsnjhfeefgijlmnnonnnnnoooonmljhfeccccdfhknsvy|}~~}|yvspnkihfeeffghijkklmnnnnnmljhgfeffghjlmoprstuuvvwwvutsqnljhfedddefghjklmmmlkkjjjjjjkkllmmmmmnopqstvwxxxwvusqomkjihijklmnoppponnmmmnnnoonnmmmlmmmnnopqqsstttutssrponllkjjkkkkkkkkkkkkllmnnnonnnnnnnnoppqqqrrrrrrrsrrqponmkjjhhhhiijjjkklkkklklllkllmlmllkmlllppss\x7f\x8a|~xpc^[QMX`aky\x81|~\x83|nni_X]a^`honquwspruoiqrlknrjhpnfjpppsyzyzyusmhfdbacgijotstvutolphchdecfokn|~~\x7f\x83\x86zwzqjgffbcilnqvzxwutnigfcbdijlqrqpnpklps{|\x81\x89\x86\x84{wpYVTHIPYbiy\x85\x82\x85\x86{pfbYLW[Yekwyv~vnjaaXXc^lx\x80\x90\x8e\x95\x98\x87\x81p_TDB?CQZn|\x86\x91\x91\x92\x87xscVSQTV^oqw\x82\x82\x82ysq^]aXk{\x7f\x91\x9a\xa4\xa1\x8f\x8buXM>72>QZq\x84\x90\x94\x8f\x90\x7fkbVNHS_`r\x81\x82\x84\x83\x81vhfaTUabaw\x8c\x8b\x90\x9f\x9e\x90\x86\x80lURICDM\\an}\x80\x81\x7f~rdf]V]bhkv\x81xz\x7fpjgeaZdmhq\x82\x87\x8e\x95\x98\x99\x8d\x88~gdXKNKV[art|}||nhi][adhnrvurujfh]_aamn|\x8f\x85\x94\x9a\x8c\x91\x81uoTXQDTU[glv{wxwfecYYYffduuoqqncdi\\ajhpv\x85\x8d\x86\x98\x96\x8a\x90~wkZaKP_Qdiixmutcl_^aYigisnxlkn\\e_\\d^mon|\x7f\x85\x8b\x8a\x90\x8e\x81~ykaaZW[Xdggtrqunmgcd^efciljhihgfejchrlt\x83\x85\x84\x8c\x90\x87\x87\x80tra__R\\eWhoislqobldaeblfevhholiemnentsv\x80\x89\x85\x87\x92\x8c\x85\x88\x80qrhg`Zk`aqgssjufgi^fbciallaljcfgkhlvrwz}\x83\x83\x89\x85\x82\x8a}stteZjeTfkdinnedh_]__dadiehjhhegmekury|z\x8d{}\x94w~\x88ozqiq_im]llenghg]g_Xe]_f_ficgddgdlonx{z}\x80\x83~\x80\x81}\x7f\x7fwxysnkmkfgchjcgfgfagebhgdhjifiqkixwsx~|u\x81\x84s~\x80y}t\x82xl}rilfmc`g`g`ah_eeaf`ed_gahjjrpvts{rsvrxrv{qzysxs|tgwmcifi__lbbkigcil^_khaerqlnyvnvtrqpxtv\x81vy~{{tyykpqlieonbiokffnkbfkhcdlpklqtrmqzuptvwsq{vnuvytnzzmnmsm]dkdablncdkhb_cgb_gnljnttkhsqjloysitwwsluxrpilqihhiphehfihfighe^cdfgbjrnkltwplmqurqpuzwwuz{unospmloolmpqonormfhmnjflrolkqsmjkpvqqwzzqpwvqlnxxtstzwnkmrpihmqojjllib`ellfelrmggmpjghntoinuyrkqxvolpsnfelpjccinnjefiie`_fjjebhonkgjruqlmrxuppqsvusrssqnkjlnnkkmoolgefhgfffjmmlnomkkmprrqrvxxxxxzyxvtqpqrqnommonnljhghjhfgjllklmonklnpoorussuvwwwvvwusomlnmkihhjihgeeeefggfegikijllikmpqqrtvxzyywvwurpnmllljjihhijjggfghhhjjjjlmmlklmnortvvxyz|{zxvtronmkjjjllkkkjihghhhhhjihjkkiiklmoprtv{||ywvsnkhgdbchjkkjkmkheddccdfiihhjkjggfgjlpv{~\x7f\x81\x81\x7fyrmiebaaeimprssqomifdeeehkllmmnlhhhijmqv{\x80\x84\x84\x85\x82}voheb`bcfmswxwvsrmigedegknnnooomigffgkqw}\x81\x85\x86\x86\x82{ric_][]agmtwxxuqmiecabcfjlnpqpmjgfdfkrw}\x81\x86\x89\x87\x82zqib^[Y[`fntwxxuplhca`adejmopoomigeefhnw}\x83\x87\x89\x8c\x88\x80wne_[YZ^flty|}{wqlgc``adhnqstttpkigfkqv~\x84\x8b\x90\x91\x8d\x85{rh`ZWW\\cjqv{{yuoid_^^_bflpqqqqmieddflsz\x81\x87\x8a\x8a\x86\x7ftj`XSPQV^fmty{xtmhc^[Z\\aejnqstrokihhmtz\x82\x88\x8f\x93\x91\x8b\x82wnbXSQTX`iqy}}{vqjc_[\\^`ejosrsromjhgjrz\x80\x85\x8d\x93\x92\x8b\x82ypcXQORTZcltyzzxrlda^\\\\^djmpsuuqomkjlqx~\x83\x89\x8f\x8f\x8a\x81yqfYSSUVZcmtvuuvofa__[Z^djmmpssnihhfflsz\x7f\x82\x88\x89\x86\x81xslc`]^`bhkoqppmjheffegiloqrrqqplifdbbeint{\x80\x85\x88\x87\x84\x80ysnhecbcccdcddcdefgfhhhhgffddb```adipx\x80\x88\x8f\x93\x93\x8f\x89\x80vlc]YXY\\adimorqppnmjhhffffggggffggjnt{\x82\x8a\x91\x95\x96\x93\x8d\x85znd]WTUW[`dhkmnmmljhfedcccceefgghhjmrx\x7f\x87\x8e\x94\x96\x96\x92\x8a\x80ukc]YXZ]cgloqsrpoljhfffghhijjjiihhjnu{\x84\x8c\x92\x96\x97\x93\x8d\x85zof^YXYZ^cgjlmmljihffffgijjjjihffgjou|\x84\x8c\x93\x97\x98\x96\x8f\x86|pf]WTTVZ_cghjjihgeddddeeefffeddegjou|\x83\x8b\x90\x93\x93\x8f\x89\x7fth^WRPQTY_diklljigedbcceghijkkjjjkmrx\x80\x87\x8d\x92\x95\x93\x8e\x86{pe\\VSSUZ_dhklljgecaa`bceghiihgfghmv}\x85\x8e\x94\x9c\x99\x92\x8a\x7fseWTQQTYcjorrtniea_\\[_bdgjnnlhghhiq|\x87\x8e\x93\x9a\x9e\x99\x8f\x82xmaWSUZ\\bkruspmic\\Y[\\^aekoppnnnmnpv~\x87\x90\x95\x9a\x9f\x9e\x97\x8c\x81xne]\\aeimsyzupjfa\\YY]abekppmlmonnry\x83\x88\x8b\x8f\x94\x94\x8d\x83|ulc]^`acfkonid_\\WSRSW[\\_cgiffgiklov~\x86\x89\x8b\x8d\x8b\x88\x80wpid`__bfjkkjhc^ZVUSTWZ`chkkkloqqu|\x84\x8a\x8b\x8b\x8d\x8c\x88\x7fuokgc`behjiihec^ZVUUUVY\\aefimqttwz\x7f\x83\x84\x84\x84\x83\x81|yuqoljjjjhfec`^[YYYZZ]_bfikmqvxz|\x80\x83\x84\x83\x82\x81\x81\x7fzxvvvsrqomjhecaa``adefijlmopsuxxz~\x80\x83\x85\x84\x84\x83\x82\x82~|zwvroljigdb``a``acehklmopqtwy|}~}}|{ywusqpnnkigda^\\\\[[]^`bceiklnoprsttuvvvwuttsqpomkhfca`__^]]]`cdffhjkmopqsstuvvwxvvututqnkiiiigcbacdffdcdfjmmnnoqtvwwvutvwwvtrrsrqnlhedddcabccdgkllkjlmorsssstuutssrrrqnkkjjigfggiiihggijkllnqrsttsrststuvvvtsssqoonmlkklmnoooononoopqrqpqsuuutrppqqrqooponnnljhikmnllkkmonlkjlnoonlklmmlklmmmmlkkklljhggghjjigggijkkjijlmnlkklmnpqpmkkkkmmmmlkmnonmkkllmnoonnnnopqqrrqsssssqpqqqrqonnoopqponnopqrrqoooqrqpooopqrsrppppoppooopqponoppoomlkmooonmllmoonmllmonmllkklmmlllmmnooponlkiijklllkjklmnlkjkkmnopponmnoomlkkmmmomkklmoonmljkmnnonmmoqqpooqqqqponnnnnoppqpponmmmlkjjkkklmmllklmmmlklnppomlkjlllllllllmmllllllkkkjklmmmmlmnpqrqppqpnllkkkkkmlllkkjjkkkklmopopqrqqrrtttsrqqqomjhiijjkkkjjjiijjlnpqstuvxxxxvtrpnmmlkiiijkjihhhhiijjijkkmnpqrtwy||{{ywtpmifdcbdfgiiihhihhihghhjjkklorvy|\x7f\x81\x81\x81\x7f|wqkfb^]]_cfiklmljhecbaaacfghkpv|\x81\x84\x87\x89\x8a\x89\x85\x80xog`\\[\\^agmruutrmieb`__behiklmnnptwz}\x83\x88\x8b\x8c\x89\x85\x7fwnf_[XY]biortutrmhc_][\\_`begjlnoptz\x7f\x85\x8a\x8d\x90\x91\x8e\x88\x7fvld]ZWY\\`fknoonkhd`]\\]_begjlnonnorw{\x80\x85\x8a\x8e\x8f\x8d\x88\x81yoga[XXY[^acdefeeffghjlmoonnnmkiiginsz\x81\x89\x8f\x93\x95\x92\x8d\x86|sjb[WTSSTVWZ]`cfiklnonnmkjhgfefgkqw~\x86\x8d\x95\x98\x9a\x97\x91\x89~ri`XTRRTVX[^acgjnqstutrolifeddefhjmqv|\x84\x8a\x90\x95\x98\x97\x93\x8c\x83ynd\\VROOQSW[`einqsuutrpnljihggghiknrw}\x83\x8a\x90\x94\x96\x96\x92\x8c\x83ypg_XTQPPRTW[_cgkoruvvutqomjhffddeefjnu|\x83\x8b\x92\x96\x98\x96\x90\x87}qf[SMIIKNSX^cimquvwxvvusrqonlkigeegilqw}\x84\x8a\x8e\x90\x90\x8c\x85~vlc[UPLKLMQV[agnswz|{zvsokhecbabcefghjlnptx|\x80\x83\x85\x86\x84\x81}xrnid`^\\[[\\^behlnprrrqponlkjjjjklnoqrrtuuvvvvvvuutsrqqpnnmlkjhhghhhijklmmnnnonoooopqrrsrqpponmlkiiihhhhhiijjjjkkkjjjkkkklllklllllkkkkklkjjjiiijjklllmlmllmmmmmmmnlmllmmmmmopqqrttttsrqqponoooooppppppppoononnnnnnnnmnnonnnoooopooooooooopoppqqqqppspsrorturroqpqonmmnooooopppooonnnmmmmmmmmlllllllmmmmnmnlnmmmmllkllllmmnnnnonoooopopmpnomomnmnmnnpopopopqmomnmllnolplpmoonokqhsgoimjmjmjmkmlmlnkpjpinhmimjkljniohqhnlkpinmiqgrhnnjnlkoilloeucvgoniojmhkmfmgpeuewcvionlqjqnolugwewb|W\x8dG\x92L\x85arqf{a|ayeujnpkqjnmisesfsisa~^xeqojpqoiwgz`~fukoukmxd~]\x82bwerqgpmviqm|]zjrkg{dismtX\x8fO\x93=\xa1Oyna\x8fC\x98I\x9aA\x92V\x82fizojtc\x84V\x85R\x90O\x7fjg\x80Zvmoe\x7fQ\x97H\x85^{hioyla\x81gum_\x93Q{d|d{O\x95YzaxssZv\x82Zw]\x89_n`\x96Nwns{L\x8dd\x80P\x91b{il\x8bR\x81k}axjzr^\x8abzsi\x88_zporave~Uypu\\uvo^jzkXk{cd_\x88hWvmzJzfuRiyed`\x89M~C\x99:\x8eG\x83_mklmh|E\x97A\x87J\x7fcuQ\x82qalixnO}smT}c\x88Gszkgbn\x89Wfrwkb]\x95`ijx\x82_]\x83zffj\x8aeco\x85fr_\x8akij\x80pkez{^ol\x81jax~jmjt|Utmohkk{`tmmpqksisisjqlqnkkop]rgtekqyfkmqq^ftljkp{nipmnjfopplqzqqm|mmdmmoerwwrlyoomhmmlosoyiuspjkqrnosuwjmnnohiorojpwnoknqcqtrqovujgrtudpyroevxkejpv`kurlemwnbimti]mqkd`kujkjz\x84ttu{wgejloahupnhkofbbghgdgjjhehidachjkiilpkqs\x80~x{z|mckib_eumkpqxkabmlg^jqm``dhe[Zbkmfgnsoffrwtzz\x85\x85~yvxulZbkkgklxqjoihffhoeknnikbkkhhhksmprqurjnlsxv~\x85\x81\x7f{xumiiljilmljjnlebjonlnptidillbaktqknuwrhdfinmp}\x87\x88\x7fuzzwjbirleapqqlhnpjhipunpstqlipolgkswrruxxpnpqnknu|{x\x7f~}rnqvogeqoohhorlffmsqnlnnjcfhjfdhnmmomkjhmlhhnw|wwx|xokorrkjmpnicfjljdfiookffjhcaglnmkknonkiiloqsw|{yvwxrplknoqqnnljkkggdghilnijjjieaceikmnnklloonnoqtuvwwutsprpmkklnkfadjjeaclmkgcfeffba_dnqrmkqrqmjlpsxyurry{wqnrvuolnnmhfgfhijnprnjhghheehmstuttrpnprux|}{xwy||wvwxwttttplknommoqronmkihikkjlpttpopqqqstvwvvvvvrppsxxqifikhd`^bfkmjgcdfeb`_adhmomjfdhmnnorvwtpopsusqllmmmhdaacehkoomhdefiiecfjpqrqpopqstuw{}|zvuvxwurnkklkkjhfghkmlidbceggffgjlnnllkpv{|wsrw{|{upprttpkhimnmjfglqqkebbehjhecflqromnswwwsrtx}~xsprutnhccfjljdaaejie`^aeghgfgjloopsvyyvuuvwwusqrttsoliijkljhgjlkigedefhihghjlnpqsuw{\x7f\x80\x80|yxvvuspmlmoonkhhgijihffgikjgdcgknomlnqv|\x7f~{wwyxuqmjijkmmmljgeccdfffgikkkkjhfgghjjmqvz}\x7f\x80\x7f~|xtoifegggikmnmjgdcccbaadinrplhfgjmmlkmrx\x7f\x84\x88\x88\x85\x81|wqkfa^_aemrutqnljifb``dkrvvrnkjmnmkhgks~\x88\x8f\x90\x8c\x85}wqjb[URU^isy{yuqnkhd_]]bhouwuqmkjkkheejs\x7f\x8a\x90\x90\x8b\x82xogaZURRV^ju}\x7f~ztngb]ZY]bhosutqnlkihfdflw\x85\x91\x97\x95\x8d\x81ti`YUQPRYdr\x7f\x87\x89\x85}tjc]ZY[_dksx{yvrmifbadkw\x85\x91\x99\x99\x91\x85vh]TOLLPXdr\x80\x89\x8d\x89\x81wlc\\XXZ^elsxywsnieb``dm{\x8b\x97\x9e\x9b\x91\x82p_RIDBDJVev\x86\x90\x93\x8f\x84wkaZVVX^dmu{}|xrkd_^bly\x87\x93\x99\x99\x91\x85ueWKCACJWgx\x86\x8e\x90\x8b\x83wj`XSRV\\dlsxxvrlga][^gu\x85\x93\x9b\x9b\x94\x85tcTICADLXi{\x8c\x97\x9c\x99\x8f\x81re\\VTVY_fnuz|{vnfbcjw\x84\x90\x98\x98\x91\x85vgZOIFHOZhw\x85\x8f\x92\x8f\x87{ocZTSUZ`flprrokgb`bit\x80\x8b\x91\x91\x8b\x80sfZPKHKR]ky\x85\x8d\x8f\x8c\x84ymbYTSUZ_fjmooqv}\x87\x91\x98\x9b\x98\x91\x85wh[NE@@GRbs\x83\x8d\x92\x8f\x86zncYTRRV]dkqvyzzzz{}\x7f\x81\x81~ztnhc^YUTUZairy||zvrnjfb``bir~\x89\x91\x95\x94\x8d\x83wlc[VRQRV]envzzvoga^^bgmsy~\x81\x82\x80|vohc``cglqtutrqpomkhebaadiouy}}|yvrmiea^]]^`bdeffggghikot{\x85\x8f\x97\x9b\x9b\x96\x8c\x7fqbVMHHMT]fmrsqmf^YUUY_gov|\x80\x83\x85\x86\x86\x85\x83\x80~|zxvtpkfa\\XUUUWY]bhntz\x7f\x83\x86\x88\x89\x87\x84\x7fxrmifdccccdcba_]\\[\\_cinty}\x7f\x80\x80~|zwtrrrsssrpnkgc`][[]`dhknqstuvutrpnmmmmmlljigfdccdfhjlnoqstvwwwvutsqponmllkjhggfgghikmoqstuvwxyyyxwusqomljhgeffghjkklmnnooopqstuuvutsqpomkigffggghijklnopqqqqrrrrqonmlkjjiijjkllmoopqrrrrsssssssrqqqppponmllkkkkjkklmnopppqqrrqqqppoonmkjhhggfffgghhijjkkkllmmmlllllkkkkkkklmmooppqqppppppppppppoooooonmmmmmmmmmmmmmmnnnopppqqppooonmmllllllmmmnnoopppppppqqqqqpppoopppppooonnmmlkkkjjiiihiiiihiijjkkkkkkllmmnnnoooppppooonnnoonnonnnnnnooooooooooooooonnnmmllllllllllmmnnnnnnnooooppooooonnnnnmmmmmmmmmmmnnmmmmmmmmmnnnoooooonnoooooooooooooooooooooooooononooooonnnnnmmlllllllllkkklkkkkkkkkkkkkkkkkkkkklkkllllmmmnnnoooooppppppppopoooooonnnnnnnnooonnooonoonnnnnnooonoooooooooooonnnnonnnnonnnmmmmmmmmmllllllkkkkkkkkjkkkkklllmmmmmnnnnnnnnnnmmmllllllllllmmmmmmmmmnnmmnnnnnnnnnnnnnnnnnnnnnnnnoooooooooonnooooonnooonnnnooonoonoonnnnnnoooooppqqqqqqqqpppooooonnmmmmmmllllmlllllllmmmmmmmmmmmmnnnmmmmmmmmlllmmmmmmmmmmmmnnnnnnnnoonnnmmmmmmllllllllllllmmmmmnnnnnnooooooooooooooonnooooooppppoooooooonnnnnnmmmllmllmmmmmnnnnnnooooooooooooppooooonnnmmnmmmmmmmmmmmmnnnmmmmnmmmmmnnnnnnnnnnnnnmnnnnmmmmmmmmmnnnnnnnmnmmnnmnnnmmmmlllllllllllllllllllmmmllmmmmmnnnmnnnnnnnmmmmnmmmmmnnnmnnnnnmnnnnnnmmmmmmmmnnnnnooooppppppppppppqqqqqppqqqpppppppoooooooonnnnonnmmmmmmmmmmmmmmmlmmmmmmmmlmlmmlllllllllllllllmmmmnnmmmmmmmmllllklkkklllmmmmmnnnnnnnooooooopppppppppppponnnmnnmmmmnnnnnnnnnnnnnononnnnnnnnmnnnnmmmmmmlmmmlllllllllmmmlllmmmmllllmmllllmmmmmmnnmnnnnooonnnnnnnnmmmmnnmmmmnmmmmmnnnooooooopooooooppoopoooooppppppppppppppooooooonnnnnnmmnmmmnnnnnnnnnnnnnnnoooooonnnnnmmlllllllllllllllllllllllllklllmlllmmmmmmmmmmnnnmmmmmmlmmmmmmmmmmmmllmmlllmmmmmnnnoooooooooooooooononnnoooooooooonnnnooonnooooooooooppooooonnnnnnnnoonnnnnnnnnnmnnnnnnnnoonnnnnnnnnmmmmmmmllllmmmmllmmmmmmmmmmnmmmmmmmmmmmmllllllllmmmmmnnnnnnnnnnnnnnnnnnmmmmmllmmmmlmmmmmmnnnnnnnnnnnnnnnnnmmmmmllllmlllllmmmmmmnnmmnnnnnnnnmnmmmmnmmmnnnnnnnnnnnnnmmmnnnnnnnnnnonooooooonnnnnnnnnnnnnnmnnnnnnnnnnnnnnnnnnnnnnmmnnnnnnnnnnnnmmmmnnnnnooooooonnnnnmmmmmllllllmmmmmmnnnnnnnononnnnnonnnnnnnnnnmmmmlmlllllmmnmmmmnnnnnooooooooooonnnnnnnnnmmmmmmmmmmmmnmmmnnnnnnnnnnnnnnnnnnnnnnnmnnnoooooopppppppppooonnnnnnnmmmmmllllmmmmlmmmnnnnnnooononnnnnnnnnmmmmllmmmmmlmmmmmmlmmmmmmmnmnnnnnmmmmmmnnnnmnnnnnnonoooonnnmmmmmmmmmmmmmmmnnnnnnmnnnnnnnooooooooooooooooonnmmmmmmmmmmnnnmmmmmmmnnnnnnnnnnnonnnnnnnmmmmmmmmmmmmmmnnnnnnnonnnnnnmmmlllllllllllllllmmmmnnnnnoooooooonnnnnnnnnnmmmmmmmmmmmmmmnnnnnnnnoooooooooooooooonnnmmmmmmmnnnnnnnnnnnnnoopoooooooooooonoonnmmnnnmnnnnoonnnnnnnnnnnnonnnnnnnnnnnmmmmmmmlllllllllllmlllllmmmllllllllmlmmnnnnnnnnnnnnnmmmlllllllllllllllmmmmmmlllmllmmmmmnnnnoooooopppooppppooooooooonnmmmmmllllmmmmmmmnnnmnnnnnnnnnoonnnnnnnnnmnmmnnmmmmmnnnmmmmmmmmmmmmmmmnnnnnnnnnnmmmmmllllllllmmmnmnnnnnnnnooooooooooonooooonoooooooooopppppooppppppppppooooooooooooooooopppqqpqqqqqppppoonnnmmmlllklkkkkkkllllmmmmmmmnnnnnnnooonnnnnnnnnnmmmmlllllkkkkkkkjkkkkkllmlllmmmmnnnnnooooooooooooooonnnmlllllllllmlllmmmmmmmmmmmmmnnnoopppppppqppppppppoooonnnnnnmmmmlllllllmmmmnnnoooppppppppooooonnnnnnnnmmmmnmmmmmmmmlmmmmmmmmllllkllllllllllmmmnnnoooooooonnnnmmmmlllkkkkkllllmmmmnnooooppppppppppppppoooonnmmmmlllllllllmmmmnnnoooopoooooooooooppppoooonnnnnmmllllkkllmlmmmmmmmmmnnnnnmmmnnnnnnnoooonnonnonnnnnnnnnmmmmmmmmmlllllkkkkkkllllllmnnnoooppppoooooonnnnnnnnnmnnnnnnnnoonooooooppppppppoooooooonmmmmmmmmmmnnmnmmmmmmmmmmmmmmmmmmmmmmmmnmmmmmmmmmmmlllllllklllkkkkkkkkkjkkkkkkklllmmnnoooppppppppooooooonnnnnmmmllllmmmnooppqpqqqqqqqqqrrrqqqqpppppppppppoopppppoooonnnnnmnmmmmmmmmmmmmmmmmmmmmmmmmmmmllllllllllllmmmmmmnnnnmmmmmmlllkkkkjkkkkjkkkkklllmmmmmmmmnnonnoooooooooonnmmmlllllllllmmmnooopqqqqqqqqppppooooooooononnoooonnnnnnoooonooooooonooonnnoooooooopppppoppponmmllkkkjjjjjjjjjjkkklmmmnnnnnnnoooonnonmmmllkkkjjjjjkkkkkklllmmmmmlmmmmmmnnnnnnnnnnnnnnnnnmmmmmnnnnnnnnnooooooooooooooooooooooopopppppppppppppppppooonnmmmmllllmmmmmmmmmmmmmmmmmmmlmlmlllkklkkkkkkjjkkjkjkkkkkkkkkkkllllllmmmmnnnnnnooonooooooppppqqrrrrrsrrrrqqrqqqpppppppppqqqqqqqqrrssssssssssrssrrqqppoonnmmmmmmmmmmnnnnoooonoonnnnmmlllkkklkkkkkkkkkkkkkkkllkkkkkkkkjkkkkkkkkkkkllllllmmmnnnnnnnnnnnmlmlllllkkkllllmmmmnnnnnnnonoooonooooooonooonnnmmnnnmnnnnnnoooopppoppppooppopoooooonnnnmmmmmmllkkkkklllllmmmmnmnnnnnnnnnmmmllllkkkkkkkkkkklllllmmmmmmmmnnnnnmmmmmmllllllllmmmmnnnnnooppoppppppppoooonnnnnnnnnnnnnooooppqqrrssssssssssrrqqqppoooonnnnnnnnnnooopppppppoooonnmmmmllkkkkkjkkkkkkkllllllllllllkkkkjjjiiiiiiiiiiijjjkklmmnnnooooopopooonnmmnnmmmnnnnnooooooooooooooooppppppppppppppoooooonnnnnmnnnmmmmmmlllllkkkkjjjjjjjjkkkkkkllllllllllkkkjjiiiiiiiijjjkkkllmmmmnnnooooopppppppppppppooooppppppppqqppqqqqrqrrrrrrqrrqrqrqqqqqqqqrrrssssstttssrrrqqpoonnmmlllkkkkkjkkkllllmmmmnnnnnnonnnnnnnnmmmmmllllllllmmmmmnnnnoooooooooooonnnnnnnnmmmmmlllkkkjjjjjjkkkkllmnnooppqqqqqqqqqpponnmmmlllllllllllllmmmmmnnnnnnnnnmmmlllkkkkkkkkkllllmmmmnnnnnnnnnnnnnmmmmlllllllllmmnnoooppqqqqqqqppppooooooonnnoooooooooooononnnnnnonnonnnnnnnnnnnnmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmnmnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn'
chunks = split_audio_bytes(audio_bytes, sample_rate=8000, sample_width=1)

In [8]:
chunks

[b'oooooppooooonooonooonnooonnoonnoonoooooonnooonnoonnnnnnnnmnmlmmmmmmmmmmmmmllmmmlmlmmmmmmmmmmmlmlmmllllllllllllllllllllllllllllllkllllllllmmlllmllmmmmmmmmmmmmmmnmmmmmmnnmmmmnmnnmnmnnnmmmmmmmmnnnnnnnnnnnnnnoooononooooooopooopooooooppooooooooooooonnnnnmnnnnnnnnnnnnnnnmmnnmmmmmmmmnmmmnnnnnnnnnnnnnnonnnnnnnnnnmnnnmmmnmmmmmmmnnnmnnnmnnnnnnnnnmmmmmmmmmmmmmnmmmnnnnnnooonoonnnnnnnnnonnnnnoooooooopppoooopppppppppoppoppooononnnnnnnnnnnnnnnnnnnnmnonnmnnnnnonnnnooonnnnnnnnnmmmmnmmmmmnnnmmmmmmnnmmmmmmmmlllmmmmlllmmlmlmlllmmmmmllllllmllllllkkllkklkllllllllllmmmmmmlllllmlllmmmllmllmlmmmmmmmmmmlmnmmnnnnnnmmmmmmmmmmmmnnmnnnoonnooooooonooooopooooooppoopppoopppooppopppopppoopoopoooooooooooooooooooooooooooponnoooonoonnnoonnnnnmmnnmmmnmmmmnmmnmmmmmllmmllmlllllllmmlllmlmmmllmlmmmmmmmmmmmmmmnnmmmmlmmlmmlllmmmlmmmmmnnmmmmmmnmmnnoooooonoooonoonooonnnnooonnnooonnnnnooonnnnnoooonnopoooooonnoonnoopoooooooooonnnnnnnnnoonnnnnnnnnnnnnnnnnnmmmnnnnmmmnmmmmmmnnnnnnmmmmmmmnmmmmmlllmllmllmmmmmmmmmmmmnmnnmmmmmnnmnnnnmmn

In [10]:
import wave

with wave.open('output_wave2sec.wav', 'w') as wf:
        # Set parameters
        n_channels = 1  # Mono
        sampwidth = 1  # 2 bytes (16 bits)
        wf.setnchannels(n_channels)
        wf.setsampwidth(sampwidth)
        wf.setframerate(8000)
        
        # Write the data

        wf.writeframes(chunks[0])

# Test endpoint of call_logs

In [2]:
import requests

# Define the URL and parameters
url = 'http://localhost:8001/call_logs/'
params = {
    'caller_id': '1293absdhi',
    'event_type': 'ASR_Output',
    'event_detail': 'Yes'
}

# Send the POST request
requests.post(url, params=params, headers={'accept': 'application/json'})

<Response [200]>

In [2]:
import pandas as pd

df= pd.read_csv('clients.csv')
df

,sn,name,number
0,1,Abhinandan Shrestha,9868205040
1,2,Amit Jung Himali,9805816686
2,3,Landline,5000
3,4,XYZ,123456
4,5,ABC,456789
